# Summary

Foreknowledge of water availability is a crucial component of logistics for water utility companies, as well as in an increasingly water-scarce world in general. Here, we develop methodologies for predicting the water levels of nine Italian water bodies. For each water body, the following process is followed: first, an analysis of the available data, and especially the development of strategies for dealing with missing data. Then, the implementation of two machine learning learning algorithms - XGBRegressor and LSTM Neural Networks - to determine which variables most influence water levels, and predict future values of the water levels. The LSTM implementation includes multiple time-step predictions, so the performance of the implementation can be assessed. Each target variable will be predicted 30 days in advance for the entire test set, which is usually 365 days. An interactive widget with a slider is used to visualize these predictions versus their actual values, as well as the mean absolute error (MAE) and root-mean-square error (RMSE), for each of the 30 days. The widget is not interactive when viewed in a static Kaggle Jupyter Notebook, but is interactive when the notebook is actively running and the results are being plotted. Due to this shortcoming, a dataframe containing the MAE and RMSE for each of the 30 days is also displayed. 

Both the XGBRegressor and LSTM models are easily modified to predict any number of days (only limited by the amount of training data), and is suited for handling stationary data by differencing, using walk-forward validation, using a simplified attention mechanism, training on synthetic data, and other options. The conclusion will include an example of how to predict water levels for one of the nine water bodies in real time.

# Introduction


First, we want to understand the basic model underlying the physical processes that determine how much water is available at any given moment in time. We can use the following equation to achieve this goal. For any given day, 

current water level = prior water level + water in - water out

$$w_{current} = w_{prior} + w_{in} - w_{out}$$ 

To make actual predictions easier in the code, we will use the same formula in a different manner:

next water level = current water level + water in - water out

$$w_{next} = w_{current} + w_{in} - w_{out}$$

Now, we want to analyze each of the components of the right hand side of the latter equation. 

The current water level: for any given day, we know what the water level is.

The amount of water inflow: this is a function of rainfall, but with some complicating factors:
    1) not all rainfall reaches the water body immediately
    2) not all water in drainage basin reaches the water body; some is lost to evaporation, as well as the two main consumers of water (at least in the United States), power production and agriculture, as well as municipal purposes. 
    
In the case of power production, it is important to note that water is only consumed by thermoelectric power plants - i.e. those that require water for cooling. Furthermore, the hotter the water is, the less efficient is the cooling, leading to hotter water coming out of the power plants, which decreases the efficiency of downstream power plants, and increases the total amount of water consumed.

The amount of water outflow: in some cases this quantity is given, but in the cases of lakes/rivers, must account for evaporation. In the case of rivers, which are defined by their flux, the "water out" variable only includes water that leaves the watershed upstream of the point of measurement. In other water bodies, it also includes water that leaves the water body after arriving.

This leads to a tricky problem: how do we calculate when rainfall reaches a water body? We must take into account
fluxes of rivers, and the sizes of watersheds. Furthermore, there can be many nonlinearities involved in this calculation. In addition to fluctuations in the amount of water used for agriculture, there will also be nonlinearities introduced by the release and filling of dams, holidays, vacation periods, tourism, music festivals, etc. 

For these reasons, rather than approaching the problem from a physical perspective, we will approach the problem from a machine learning perspective. If we acknowledge the fact that it is very difficult to construct a physical model that will predict water levels accurately, the next best thing we can do is gather data and try to learn from the past in order to predict the future. Indeed, a well-cited study found that

>ANNs \[Artifical Neural Networks\] used the initial water level measurements, production well extractions, and climate conditions to predict the final water level elevations 30 d into the future at two monitoring wells. A sensitivity analysis was conducted with the ANNs that quantified the importance of the various input predictor variables on final water level elevations. Unlike traditional physical‐based models, ANNs do not require explicit characterization of the physical system and related physical data. Accordingly, ANN predictions were made on the basis of more easily quantifiable, measured variables, rather than physical model input parameters and conditions. 
> - [A neural network model for predicting aquifer water level elevations](https://ngwa.onlinelibrary.wiley.com/doi/abs/10.1111/j.1745-6584.2005.0003.x)

Several other studies have had success taking this approach as well: 

- [Predicting piezometric water level in dams via artificial neural networks](https://link.springer.com/article/10.1007/s00521-012-1334-2)

- [Artificial neural network approach to flood forecasting in the River Arno](https://www.researchgate.net/publication/248952964_Artificial_neural_network_approach_to_flood_forecasting_in_the_River_Arno)

- [Predicting flood susceptibility using LSTM neural networks](https://www.researchgate.net/publication/346777911_Predicting_flood_susceptibility_using_LSTM_neural_networks)

- [River flood forecasting with a neural network model ](https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/1998WR900086)

Each of the nine datasets for which we are making predictions contains some combination of rainfall, temperature, river level, lake level, depth to aquifer, flow rate, and volume of water extracted from the water body.  

In order to account for non-linearities in the data, we will be using non-linear predictors, in particular XGBRegressor and LSTM Neural Networks.

In order to capture the effects of the non-linearities, the following feature extraction is proposed: we make dummy variables for every week and month of the year, dummy variables for temperature brackets, and then make cross terms between 1) the time dummy variables and the temperature dummy variables; 2) the time dummy variables and other variables; 3) the temperature dummy variables and other variables. 

The idea behind making these dummy variables is for the machine learning algorithm to learn which variables are most important on the water level on any given day, and how to weigh those variables to predict that water level. We will see in the "Feature Importances" sections that this method does in fact capture relationships between the data that would not otherwise be apparent.


# Choice of Machine Learning Algorithms

We tested a number of different machine learning algorithms to solve this problem: Lasso regression (a type of linear regression where the least important variables are eliminated from the regression), k-Nearest-Neighbors, Support Vector Regression, XGBRegressor, Feed-forward Neural Networks (FNN), and Long-Short-Term-Memory Neural Networks (LSTM). Of these, XGBRegressor and LSTM gave the best results, and so they are the ones we use in this notebook. 

LSTM Neural Networks are a type of recurrent neural network which are well suited for predicting sequences of data. For this reason, they are very popular in time-series analysis. However, LSTM networks are not well suited for determining which features are the most important for predicting the target variable(s). For this reason, we will use XGBRegressor to determine the most important features - including variables in the past - that affect our target variables.

In [ ]:
# imports 
import numpy as np 
import pandas as pd 
import missingno as msno 
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
from datetime import datetime, date, timedelta
from time import time
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import sklearn.metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import LSTM
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
import sys
from ipywidgets import *

import gc
from memory_profiler import profile
%load_ext memory_profiler
from ctypes import cdll, CDLL
cdll.LoadLibrary("libc.so.6")
libc = CDLL("libc.so.6")
import tensorflow as tf

# Data Cleaning

In [ ]:
def determine_time_gaps(actual_time_series):
    """
    Determines if there are any missing dates in the dataset
    
    Args: 
        actual_time_series - the actual time series from the dataset
    
    Returns:
        correct_days[~mask] - the correct days
    """
    
    correct_days = pd.Series(data=pd.date_range(start=actual_time_series.values[0], end=actual_time_series.values[-1], freq='D'))
    mask = correct_days.isin(actual_time_series.values)
    return correct_days[~mask]


In [ ]:
def fill_in_missing_data(df, strategy, order=1):
    """
    Fills missing data
    
    Args:
        df - dataframe with missing data
        strategy - which filling strategy to use
        order - if strategy is interpolate, the order of the polynomial; defaults to 1
    """
    
    if strategy == 'kNN':  # drop rows with incomplete data
        df[df.columns.drop('Date')] = KNN(k=10).fit_transform(df[df.columns.drop('Date')])
    if strategy == 'interpolate':  # drop columns with the most missing data
        df = df.interpolate(method='polynomial', order=order)
        
    return df

In [ ]:
# kaggle_path denotes whether or not we use the kaggle paths to file
kaggle_path = True

In [ ]:
local_paths = {
    'auser': 'Aquifer_Auser.csv',
    'doganella': 'Aquifer_Doganella.csv',
    'luco': 'Aquifer_Luco.csv',
    'petrignano': 'Aquifer_Petrignano.csv',
    'bilancino': 'Lake_Bilancino.csv',
    'amiata': 'Water_Spring_Amiata.csv',
    'madonna': 'Water_Spring_Madonna_di_Canneto.csv',
    'lupa': 'Water_Spring_Lupa.csv',
    'arno': 'River_Arno.csv',
    'xgb': './XGBregressor_Models',
    'xgb_results': './XGBregressor_Results',
    'lstm': './LSTM_Models',
    'lstm_results': './LSTM_Results'    
}

kaggle_paths = {
    'auser': '../input/acea-water-prediction/Aquifer_Auser.csv',
    'doganella': '../input/acea-water-prediction/Aquifer_Doganella.csv',
    'luco': '../input/acea-water-prediction/Aquifer_Luco.csv',
    'petrignano': '../input/acea-water-prediction/Aquifer_Petrignano.csv',
    'bilancino': '../input/acea-water-prediction/Lake_Bilancino.csv',
    'amiata': '../input/acea-water-prediction/Water_Spring_Amiata.csv',
    'madonna': '../input/acea-water-prediction/Water_Spring_Madonna_di_Canneto.csv',
    'lupa': '../input/acea-water-prediction/Water_Spring_Lupa.csv',
    'arno': '../input/acea-water-prediction/River_Arno.csv',
    'xgb': './XGBregressor_Models',
    'xgb_results': './XGBregressor_Results',
    'lstm': './LSTM_Models',
    'lstm_results': './LSTM_Results'    
}


def get_path(dataset):
    if not kaggle_path:
        return local_paths[dataset]
    else:
        return kaggle_paths[dataset]
    

Note: we import `matplotlib.pyplot` at the start of every data cleaning subsection because later use of matplotlib magic interferes with the plots shown here if these cells are executed after the matplotlib magic is invoked.

## Auser Aquifer

According to the provided description, 

> Description: This waterbody consists of two subsystems, called NORTH and SOUTH, where the former partly influences the behavior of the latter. Indeed, the north subsystem is a water table (or unconfined) aquifer while the south subsystem is an artesian (or confined) groundwater.

> The levels of the NORTH sector are represented by the values of the SAL, PAG, CoS and DIEC wells, while the levels of the SOUTH sector by the LT2 well. 

> Output: Depth_to_Groundwater_SAL, Depth_to_Groundwater_CoS, Depth_to_Groundwater_LT2

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_aquifer_auser = pd.read_csv(get_path('auser'))
msno.matrix(df_aquifer_auser);

We can see that there is a lot of missing data, including many missing rows for the target variables.

In [ ]:
df_aquifer_auser['Date'] = pd.to_datetime(df_aquifer_auser['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps((df_aquifer_auser['Date']))
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_aquifer_auser.dropna()['Date'].values[0]))

There are no missing dates. We will remove the rows with the most missing data, which are all the rows before 2011-05-17.

In [ ]:
df_aquifer_auser['Date'] = pd.to_datetime(df_aquifer_auser['Date'], format = '%d/%m/%Y')
df_aquifer_auser = df_aquifer_auser[df_aquifer_auser['Date'] >= '2011-05-17']

Now, we can proceed more easily. Let's take a look at our data so far.

In [ ]:
df_aquifer_auser.plot(x='Date', subplots=True, figsize=(30, 90));

In [ ]:
print('Number of zero-values for Temperature_Ponte_a_Moriano = ', len(df_aquifer_auser[df_aquifer_auser['Date'] >= '2017-06-05']))

There are still a few problems. First, there are many missing values for the `Depth_to_Groundwater` variables, which is a particular problem, since they are our target variables. Second, it seems that the `Volume_CSA`, `Volume_CSAL`, `Temperature_Ponte_a_Moriano`, and some of the `Depth_to_Groundwater` variables have improbable values of 0 for extended periods of time, both early on in the modified dataset, and at some points at the end of it. 

In the case of `Temperature_Ponte_a_Moriano`, we can see that upon manual inspection, the date where all the values start being zero is 2017-06-05. Furthermore, we see that this temperature variable is highly seasonal. Therefore, it is realistic to simply ascribe all of its zero-values from 2017-06-05 onwards to the value of prior years. We can do this because XGBRegressor and LSTM will make much heavier use of the dummy variables that we construct from the temperature (e.g. is the temperature between 30 - 35 degrees?), rather than the temperature itself, and since these dummy variables are much more coarsely grained then the temperature, they are much more likely to be accurate.

In the case of the `Depth_to_Groundwater` variables, we see that the missing values are sporadic; however there is a clear trend in each of them, and so interpolating the missing values is appropriate. For the `Depth_to_Groundwater` LT2 and DIEC variables, a linear interpolation seems reasonable. For the SAL, PAG, and CoS variables, a quadratic interpolation seems more suitable.

In [ ]:
df_aquifer_auser.loc[df_aquifer_auser['Date'] >= '2017-06-05', 'Temperature_Ponte_a_Moriano'] = \
    df_aquifer_auser.loc[df_aquifer_auser['Date'] < '2017-06-05', 'Temperature_Ponte_a_Moriano'].values[-1122:]
df_aquifer_auser[['Depth_to_Groundwater_LT2', 'Depth_to_Groundwater_DIEC']] = df_aquifer_auser[['Depth_to_Groundwater_LT2', 'Depth_to_Groundwater_DIEC']].interpolate(method='linear')
df_aquifer_auser[['Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_PAG', 'Depth_to_Groundwater_CoS']] = df_aquifer_auser[['Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_PAG', 'Depth_to_Groundwater_CoS']].interpolate(method='polynomial', order=2)

Now, let's take a look at our final dataset.

In [ ]:
df_aquifer_auser.plot(x='Date', subplots=True, figsize=(30, 90)); 

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_aquifer_auser) - len(df_aquifer_auser.dropna()))

Even after our data cleaning, there are 22 rows with at least one missing value. We will interpolate these missing values to make sure that we have continous data.

The interpolation for the last year of `Depth_to_Groundwater_DIEC` does not seem so good, but since it is not one of our target variables, it is probably good enough that we do not need to reduce the dataset by a whole year just because of it.

We see that for our three target variables, `Depth_to_Groundwater_SAL`, `Depth_to_Groundwater_CoS`, and `Depth_to_Groundwater_LT2`, there is some data starting on 2020-06-05 that is probably erroneous. It is very unlikely that the values suddenly became 0 for a few days and then immediately jumped back to their previous values - there was probably some kind of measuring error. When making our predictions, we will ignore the very last part of this dataset.

In [ ]:
df_aquifer_auser = df_aquifer_auser.loc[df_aquifer_auser['Date'] < '2020-06-05', :]
df_aquifer_auser.interpolate(method='linear', inplace=True)

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_aquifer_auser) - len(df_aquifer_auser.dropna()))

## Doganella Aquifer

According to the provided description, 

> Description: The wells field Doganella is fed by two underground aquifers not fed by rivers or lakes but fed by meteoric infiltration. The upper aquifer is a water table with a thickness of about 30m. The lower aquifer is a semi-confined artesian aquifer with a thickness of 50m and is located inside lavas and tufa products. These aquifers are accessed through wells called Well 1, ..., Well 9. Approximately 80% of the drainage volumes come from the artesian aquifer. The aquifer levels are influenced by the following parameters: rainfall, humidity, subsoil, temperatures and drainage volumes.

> Output: Depth_to_Groundwater_Pozzo_1, Depth_to_Groundwater_Pozzo_2, Depth_to_Groundwater_Pozzo_3, Depth_to_Groundwater_Pozzo_4, Depth_to_Groundwater_Pozzo_5, Depth_to_Groundwater_Pozzo_6, Depth_to_Groundwater_Pozzo_7, Depth_to_Groundwater_Pozzo_8, Depth_to_Groundwater_Pozzo_9 

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_aquifer_doganella = pd.read_csv(get_path('doganella'))
msno.matrix(df_aquifer_doganella);

There is a lot of missing data, including for the target variables. 

In [ ]:
df_aquifer_doganella['Date'] = pd.to_datetime(df_aquifer_doganella['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps(df_aquifer_auser['Date'])
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_aquifer_doganella.dropna(subset=['Depth_to_Groundwater_Pozzo_3'])['Date'].values[0]))

There are no missing dates. We will remove the rows with the most missing data, which are all the rows before 2012-06-01.

In [ ]:
df_aquifer_doganella = df_aquifer_doganella[df_aquifer_doganella['Date'] >= '2012-06-01']

Now we can proceed more easily. Let's take a look at our data so far.

In [ ]:
df_aquifer_doganella.plot(x='Date', subplots=True, figsize=(30, 70));

We can see that we still have many missing values. As before, we can fill in the missing values of the temperatures - which start in 2015-01-01 and end 2018-12-31 - with those of prior years, due to their strong seasonalities and the use of dummy variables, and impute the small number of missing values in `Temperature_Velletri` in the end of 2014. 

The rest of the data is a bit trickier. We can see that a large amount of rainfall early in the truncated dataset increased the amount of water in the aquifer, but that subsequent drier years led to a steady decrease, as water was withdrawn from it. Since we see the same steady decrease in the water levels once we have actual data for the `Volume` variables starting in late 2016, the logical conclusion is that the missing values for the `Volume` variables prior to late 2016 are similar to the values that are available after late 2016.

Here is the contradiction: we do not have much data on the variation of rainfall - there is only rainfall in the beginning of the dataset, and towards the end, which is where we want to have our test set. However, we only have data for the `Volume` variables starting in late 2016. So do we impute the values of the `Volume` back in time so that we have access to the earlier rainfall data, or do we drop the data up until we have the `Volume` variables available? 

The reality is that without some very in depth data engineering, we cannot come close to correctly imputing the prior values of the `Volume` variables. For that reason, we will drop the rows which do not contain them.

In [ ]:
len_filling_data = len(df_aquifer_doganella.loc[df_aquifer_doganella['Date'] <= '2014-12-31', 'Temperature_Monteporzio'])
df_aquifer_doganella.loc[(df_aquifer_doganella['Date'] >= '2016-06-01') & (df_aquifer_doganella['Date'] <= '2018-12-31'), 'Temperature_Monteporzio'] = \
    df_aquifer_doganella.loc[(df_aquifer_doganella['Date'] >= '2012-06-01') & (df_aquifer_doganella['Date'] <= '2014-12-31'), 'Temperature_Monteporzio'].values
df_aquifer_doganella.loc[(df_aquifer_doganella['Date'] >= '2016-06-01') & (df_aquifer_doganella['Date'] <= '2018-12-31'), 'Temperature_Velletri'] = \
    df_aquifer_doganella.loc[(df_aquifer_doganella['Date'] >= '2012-06-01') & (df_aquifer_doganella['Date'] <= '2014-12-31'), 'Temperature_Velletri'].values

df_aquifer_doganella = df_aquifer_doganella.loc[df_aquifer_doganella['Date'] >= '2016-10-07'] 

Notice that the `Pozzo` variables 2, 4, 6, 8, and 9 all have similar shapes and values. Of these, Pozzo 2 looks like it has the easiest values to impute (in other words, its imputed values will be the most accurate), and we will use this variable as a representative for the others. However, we must deal with an anomaly in the data that would be in our test set: 

In [ ]:
df_aquifer_doganella[['Date', 'Depth_to_Groundwater_Pozzo_2']][-26:-16]

It seems extremely unlikely for the level of an aquifer to suddenly drop from -100 meters to -112 meters in one day, remain at that level for four days, and then jump back to -100 meters immediately after. There is no indication that the Rainfall or Volume variables would lead to this happening. Likewise, we only see the same anomaly in Pozzo 8, which has a very similar distribution to Pozzo 2, but nowhere else in the data. We will treat this as an anomaly, replace these values with np.nan, and then interpolate them.

In [ ]:
df_aquifer_doganella.loc[(df_aquifer_doganella['Date'] >= '2020-06-08') & (df_aquifer_doganella['Date'] <= '2020-06-11'), 'Depth_to_Groundwater_Pozzo_2'] = np.nan

There is one final anomaly we must consider. In late 2019, there is about a month of missing data for the Groundwater variables, and during that same time period, the Volume variables drop almost entirely to zero. Since the Volume variables represent the amount of water taken out of drinking water treatment plants, it seems even more unlikely that these values were zero for an entire month. Additionally, for the Groundwater data that we do have available for those dates, the lack of any effect from the Volume variables being zero seems unusual. Compounding this with the corresponding mostly missing data for the Groundwater variables, we will assume that these variables are errors. We will set them to `np.nan` and impute them.

So in the end, we will interpolate the missing Groundwater variable values and (few) missing Temperature variable values, and impute the missing Volume variable values.

In [ ]:
groundwater_variables = ['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2',
       'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4',
       'Depth_to_Groundwater_Pozzo_5', 'Depth_to_Groundwater_Pozzo_6',
       'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8',
       'Depth_to_Groundwater_Pozzo_9']
volume_variables = ['Volume_Pozzo_1', 'Volume_Pozzo_2',
       'Volume_Pozzo_3', 'Volume_Pozzo_4', 'Volume_Pozzo_5+6',
       'Volume_Pozzo_7', 'Volume_Pozzo_8', 'Volume_Pozzo_9'] 
temperature_variables = ['Temperature_Monteporzio', 'Temperature_Velletri']
df_aquifer_doganella.where(df_aquifer_doganella.loc[(df_aquifer_doganella['Date'] >= '2019-09-25') & (df_aquifer_doganella['Date'] <= '2019-10-13'), volume_variables] != 0, np.nan, inplace=True)

Pozzo 3 seems dissimilar enough from the other target variables to make it worth predicting. There is mostly continous data for Pozzo 3, except at the end of the dataset, starting 2019-09-19. We will not try to impute these missing values - rather, we will make our test set the year leading up to 2019-09-19.

Pozzo 5 and Pozzo 7 have missing values which look very difficult to impute properly, so we will not try to predict these. However, their values are quite similar to the first group (represented by Pozzo 2), so predictions of the first group should give a good indication of predictions for Pozzo 5 and Pozzo 7.  

Like Pozzo 3, Pozzo 1 seems dissimilar enough from the other target variables to make it worth predicting. Pozzo 1 has some missing data and a large jump right around the time that we start having data for the `Volume` variables. We will drop all of the data before this jump/introduction of `Volume` variables, and then impute any missing values. 

In the end, we will predict `Depth_to_Groundwater_Pozzo_1`, `Depth_to_Groundwater_Pozzo_2`, and `Depth_to_Groundwater_Pozzo_3`.

In [ ]:
df_aquifer_doganella.loc[:, groundwater_variables + temperature_variables] = fill_in_missing_data(df_aquifer_doganella.loc[:, groundwater_variables + temperature_variables], 'interpolate', order=1)
df_aquifer_doganella[df_aquifer_doganella.columns.drop('Date')] = KNN(k=10).fit_transform(df_aquifer_doganella[df_aquifer_doganella.columns.drop('Date')])
df_aquifer_doganella.drop(columns=['Depth_to_Groundwater_Pozzo_4',
       'Depth_to_Groundwater_Pozzo_5', 'Depth_to_Groundwater_Pozzo_6',
       'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8',
       'Depth_to_Groundwater_Pozzo_9'], inplace=True)


Let's take a look at our final dataset. Remember that we will not be using the last portion of `Depth_to_Groundwater_Pozzo_3`, since the amount of missing data there, and the ensuing interpolation, seem unsuitable for the problem. 

In [ ]:
df_aquifer_doganella.plot(x='Date', subplots=True, sharex=True, figsize=(30, 50));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_aquifer_doganella) - len(df_aquifer_doganella.dropna()))

There are no missing values remaining.

## Luco Aquifer

According to the provided description, 

> Description: The Luco wells field is fed by an underground aquifer. This aquifer not fed by rivers or lakes but by meteoric infiltration at the extremes of the impermeable sedimentary layers. Such aquifer is accessed through wells called Well 1, Well 3 and Well 4 and is influenced by the following parameters: rainfall, depth to groundwater, temperature and drainage volumes.

> Output: Depth_to_Groundwater_Podere_Casetta

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_aquifer_luco = pd.read_csv(get_path('luco'))
msno.matrix(df_aquifer_luco);

In [ ]:
df_aquifer_luco['Date'] = pd.to_datetime(df_aquifer_luco['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps(df_aquifer_luco['Date'])
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_aquifer_luco.dropna()['Date'].values[0]))

Our target variable is `Depth_to_Groundwater_Podere_Casetta`. We will first filter our data for this variable. The first date that `Depth_to_Groundwater_Podere_Casetta` is available is 2008-02-21, and the last date it is available is 2019-01-12.

In [ ]:
dates_target_is_available = df_aquifer_luco[~df_aquifer_luco['Depth_to_Groundwater_Podere_Casetta'].isna()]['Date'].values
df_aquifer_luco = df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2008-02-21') & (df_aquifer_luco['Date'] <= '2019-01-12'), :]

In [ ]:
df_aquifer_luco.plot(x='Date', subplots=True, figsize=(30, 70));

This is an interesting dataset. It seems reasonable to use an interpolation for the values of our target variable, `Depth_to_Groundwater_Podere_Casetta`. There is only a bit of the data missing for the target in the last year, which will be our test set; however, given the smooth trendline of the data, it is reasonable to interpolate it, even in the test set.

The first question is, should we filter the data for the `Volume` variables, or should we drop them, so that we can use much more of the rest of the data? Since the `Volume` variables are probably strong predictors of our target, and we have around 4 years worth of such data starting 2015-01-01, we will filter for them.

Next, there is very little data for `Temperature_Siena_Poggio_al_Vento`, so we will drop this variable. This leaves us with trying to estimate the anomalous zero values of `Temperature_Mensano` and `Temperature_Monteroni_Arbia_Biena`. As usual, we can estimate these anomalous values with their past/future values (but not future values from the test set).

We can drop the non-target `Depth_to_Groundwater` variables, since there clearly is not enough of that data for both training and testing. 

What remains to be dealt with are the `Rainfall` variables. We will drop the following variables due to lack of data: `Rainfall_Siena_Poggio_al_Vento`, `Rainfall_Ponte_Orgia`, and `Rainfall_Pentolina`. We will impute the remaining values where they are still missing.

In [ ]:
# filter rows by Volume variables
df_aquifer_luco = df_aquifer_luco.loc[df_aquifer_luco['Date'] >= '2015-01-01']
# drop columns with not enough data
columns_to_drop_luco = ['Temperature_Siena_Poggio_al_Vento', 'Depth_to_Groundwater_Pozzo_1', 
                        'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4',
                       'Rainfall_Siena_Poggio_al_Vento', 'Rainfall_Ponte_Orgia', 'Rainfall_Pentolina']
df_aquifer_luco = df_aquifer_luco.drop(columns=columns_to_drop_luco)


In [ ]:
# determine start and end dates of Temperature_Mensano all-zero values
df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2015-01-01') & (df_aquifer_luco['Date'] <= '2015-10-13'), 'Temperature_Mensano'] = \
    df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2017-01-01') & (df_aquifer_luco['Date'] <= '2017-10-13'), 'Temperature_Mensano'].values

df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2016-10-07') & (df_aquifer_luco['Date'] <= '2016-11-21'), ['Date', 'Temperature_Monteroni_Arbia_Biena']]

# trying to replace these first three sets all at once with .where or .mask proves to be very buggy, so just do each group manually
df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2016-10-07') & (df_aquifer_luco['Date'] <= '2016-10-12'), 'Temperature_Monteroni_Arbia_Biena'] = \
    df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2017-10-07') & (df_aquifer_luco['Date'] <= '2017-10-12'), 'Temperature_Monteroni_Arbia_Biena'].values

df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2016-10-22') & (df_aquifer_luco['Date'] <= '2016-11-02'), 'Temperature_Monteroni_Arbia_Biena'] = \
    df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2017-10-22') & (df_aquifer_luco['Date'] <= '2017-11-02'), 'Temperature_Monteroni_Arbia_Biena'].values

df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2016-11-17') & (df_aquifer_luco['Date'] <= '2016-11-21'), 'Temperature_Monteroni_Arbia_Biena'] = \
    df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2017-11-17') & (df_aquifer_luco['Date'] <= '2017-11-21'), 'Temperature_Monteroni_Arbia_Biena'].values

# last set
df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2017-05-19') & (df_aquifer_luco['Date'] <= '2017-06-06'), 'Temperature_Monteroni_Arbia_Biena'] = \
    df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2016-05-19') & (df_aquifer_luco['Date'] <= '2016-06-06'), 'Temperature_Monteroni_Arbia_Biena'].values

df_aquifer_luco['Depth_to_Groundwater_Podere_Casetta'].interpolate(method='polynomial', order=2, inplace=True)
df_aquifer_luco[df_aquifer_luco.columns.drop('Date')] = KNN(k=10).fit_transform(df_aquifer_luco[df_aquifer_luco.columns.drop('Date')])


In [ ]:
df_aquifer_luco.loc[(df_aquifer_luco['Date'] >= '2017-05-19') & (df_aquifer_luco['Date'] <= '2017-06-06'), ['Date', 'Temperature_Monteroni_Arbia_Biena']];

Let's take a look at our final dataset.

In [ ]:
df_aquifer_luco.plot(x='Date', subplots=True, figsize=(30, 50));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_aquifer_luco) - len(df_aquifer_luco.dropna()))

There are no missing values remaining.

## Petrignano Aquifer

According to the provided description, 

> Description: The wells field of the alluvial plain between Ospedalicchio di Bastia Umbra and Petrignano is fed by three underground aquifers separated by low permeability septa. The aquifer can be considered a water table groundwater and is also fed by the Chiascio river. The groundwater levels are influenced by the following parameters: rainfall, depth to groundwater, temperatures and drainage volumes, level of the Chiascio river.

> Output: Depth_to_Groundwater_P24, Depth_to_Groundwater_P25

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_aquifer_petrignano = pd.read_csv(get_path('petrignano'))
msno.matrix(df_aquifer_petrignano);

There is plenty of data to work with here. The Rainfall and and Temperature variables will almost certainly be useful for our predictions, so we will drop the early rows where they are missing, which are all the rows up until 2009-01-01. 

In [ ]:
df_aquifer_petrignano['Date'] = pd.to_datetime(df_aquifer_petrignano['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps((df_aquifer_petrignano['Date']))
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_aquifer_petrignano.dropna()['Date'].values[0]))

In [ ]:
df_aquifer_petrignano = df_aquifer_petrignano.loc[df_aquifer_petrignano['Date'] >= '2009-01-01', :]

In [ ]:
df_aquifer_petrignano.plot(x='Date', subplots=True, figsize=(30, 30));

There are still a few problems with the data. We can see that starting in mid 2015, there is anomalous data for `Temperature_Petrignano` and `Hydrometry_Fiume_Chiascio_Petrignano`, where both these variables are all 0 for several months, starting 2015-04-25 and ending 2015-09-21. As with prior missing temperature data, we can use values from the year prior to replace to anomalous values of `Temperature_Petrignano`. In the case of `Hydrometry_Fiume_Chiascio_Petrignano`, we will impute the missing values.

Additionally, we can see that there is some unusual data in 2019 for `Volume_C10_Petrignano`, starting 2019-06-28 and ending 2019-08-21, as well as in 2011, where some of the values suddenly become zero and then return to values close to their prior values. It is not clear whether these values are errors or not, since this variable represents the volume of water taken from the drinking water treatment plant. It is possible that these values were in fact zero because of some distruptive event. Since it will probably not affect our model too much, we will impute these values.

There is a small amount of data still missing, some of it in our target variables. These gaps are small, and the values around the gaps are close to each other, so we will impute these values as well.

In [ ]:
# replace the anomalous temperature with the temperature of the previous year
df_aquifer_petrignano.loc[(df_aquifer_petrignano['Date'] >= '2015-04-25') & (df_aquifer_petrignano['Date'] <= '2015-09-21'), 'Temperature_Petrignano'] = \
    df_aquifer_petrignano.loc[(df_aquifer_petrignano['Date'] >= '2014-04-25') & (df_aquifer_petrignano['Date'] <= '2014-09-21'), 'Temperature_Petrignano'].values
# interpolate the missing values
df_aquifer_petrignano = df_aquifer_petrignano.interpolate(method='linear')
# replace the anomalous Hydrometry_Fiume_Chiascio_Petrignano and Volume_C10_Petrignano values with np.nan 
df_aquifer_petrignano.loc[(df_aquifer_petrignano['Date'] >= '2015-04-25') & (df_aquifer_petrignano['Date'] <= '2015-09-21'), 'Hydrometry_Fiume_Chiascio_Petrignano'] = np.nan
df_aquifer_petrignano.where(df_aquifer_petrignano['Volume_C10_Petrignano'] != 0, np.nan, inplace=True)
# fill in the missing values
df_aquifer_petrignano = fill_in_missing_data(df_aquifer_petrignano, 'kNN')

Let's take a look at our cleaned data set.

In [ ]:
df_aquifer_petrignano.plot(x='Date', subplots=True, figsize=(30, 30));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_aquifer_petrignano) - len(df_aquifer_petrignano.dropna()))

It seems there are 25 rows with at least one missing value, even after our data cleaning. Let's find out what's going on.

In [ ]:
idxr = df_aquifer_petrignano['Date'].to_frame().isna().any(axis=1)
missing_dates = df_aquifer_petrignano['Date'][idxr].index
print('missing dates indices: ', df_aquifer_petrignano.loc[missing_dates, :].index)
display(df_aquifer_petrignano.loc[4900 : 4910, :])

It looks like there are some missing dates, where the `Date` values are all `NaT` and the variables are all 0. Let's try to set these variables to `np.nan`, replace the dates, and then interpolate the values.

In [ ]:
df_aquifer_petrignano[df_aquifer_petrignano['Date'].isna()] = np.nan
petrignano_time_series = df_aquifer_petrignano['Date']
petrignano_correct_days = pd.Series(data=pd.date_range(start=petrignano_time_series.values[0], end=petrignano_time_series.values[-1], freq='D'))
df_aquifer_petrignano['Date'] = petrignano_correct_days.values
df_aquifer_petrignano = df_aquifer_petrignano.interpolate(method='linear')

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_aquifer_petrignano) - len(df_aquifer_petrignano.dropna()))

Let's see the results.

In [ ]:
msno.matrix(df_aquifer_petrignano)
df_aquifer_petrignano.plot(x='Date', subplots=True, figsize=(30, 30));

Looks good.

## Bilancino Lake

According to the provided description, 

> Description: Bilancino lake is an artificial lake located in the municipality of Barberino di Mugello (about 50 km from Florence). It is used to refill the Arno river during the summer months. Indeed, during the winter months, the lake is filled up and then, during the summer months, the water of the lake is poured into the Arno river.

> Output: Lake_Level, Flow_Rate

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_lake_bilancino = pd.read_csv(get_path('bilancino'))
msno.matrix(df_lake_bilancino);

In [ ]:
df_lake_bilancino['Date'] = pd.to_datetime(df_lake_bilancino['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps(df_lake_bilancino['Date'])
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_lake_bilancino.dropna()['Date'].values[0]))

There are no missing dates, and the most of the missing data ends on 2004-01-02, so we will take all the data after that.

In [ ]:
df_lake_bilancino = df_lake_bilancino[df_lake_bilancino['Date'] >= '2004-01-02']

Let's take a look at the final dataset.

In [ ]:
df_lake_bilancino.plot(x='Date', subplots=True, figsize=(30, 35));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_lake_bilancino) - len(df_lake_bilancino.dropna()))

## Amiata Water Spring

According to the provided description, 

> Information about the Amiata aquifer. This aquifer is accessed through the Ermicciolo, Arbure, Bugnano and Galleria Alta springs. The levels and volumes of the four springs are influenced by the parameters: pluviometry, sub-gradation, hydrometry, temperatures and drainage volumes.

> Output: Flow_Rate_Bugnano, Flow_Rate_Arbure, Flow_Rate_Ermicciolo, Flow_Rate_Galleria_Alta

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_water_spring_amiata = pd.read_csv(get_path('amiata'))
msno.matrix(df_water_spring_amiata);

In [ ]:
df_water_spring_amiata['Date'] = pd.to_datetime(df_water_spring_amiata['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps(df_water_spring_amiata['Date'])
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_water_spring_amiata.dropna()['Date'].values[0]))

There are no missing dates.

In [ ]:
df_water_spring_amiata.plot(x='Date', subplots=True, figsize=(30, 60));

We will begin by filtering for our target variables. After that, as with prior `Temperature` variables, we can estimate the missing values with those of past/future years. The `Depth_to_Groundwater` variables show a very clear trend that we can interpolate. Lastly, we will impute the missing `Rainfall` variables.

In [ ]:
df_water_spring_amiata.loc[(df_water_spring_amiata['Date'] >= '2015-02-05') & (df_water_spring_amiata['Date'] <= '2015-03-30'), 'Rainfall_Abbadia_S_Salvatore'] = \
    df_water_spring_amiata.loc[(df_water_spring_amiata['Date'] >= '2014-02-05') & (df_water_spring_amiata['Date'] <= '2014-03-30'), 'Rainfall_Abbadia_S_Salvatore'].values

df_water_spring_amiata.loc[(df_water_spring_amiata['Date'] >= '2015-07-22') & (df_water_spring_amiata['Date'] <= '2016-01-24'), 'Rainfall_Abbadia_S_Salvatore'] = \
    df_water_spring_amiata.loc[(df_water_spring_amiata['Date'] >= '2014-07-22') & (df_water_spring_amiata['Date'] <= '2015-01-24'), 'Rainfall_Abbadia_S_Salvatore'].values


depth_to_groundwater_variables = ['Depth_to_Groundwater_S_Fiora_8', 'Depth_to_Groundwater_S_Fiora_11bis', 'Depth_to_Groundwater_David_Lazzaretti']
for groundwater_variable in depth_to_groundwater_variables:
    df_water_spring_amiata[groundwater_variable].interpolate(method='polynomial', order=2, inplace=True)

df_water_spring_amiata = df_water_spring_amiata.loc[df_water_spring_amiata['Date'] >= '2015-01-01']

df_water_spring_amiata[df_water_spring_amiata.columns.drop('Date')] = KNN(k=10).fit_transform(df_water_spring_amiata[df_water_spring_amiata.columns.drop('Date')])

In [ ]:
# (~df_water_spring_amiata.loc[(df_water_spring_amiata['Date'] >= '2015-07-22') & (df_water_spring_amiata['Date'] <= '2016-01-24'), 'Temperature_Abbadia_S_Salvatore'].isna()).sum()

Let's take a look at our final dataset.

In [ ]:
df_water_spring_amiata.plot(x='Date', subplots=True, figsize=(30, 60));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_water_spring_amiata) - len(df_water_spring_amiata.dropna()))

## Madonna di Canneto Water Spring

According to the provided description, 

> Description: The Madonna di Canneto spring is situated at an altitude of 1010m above sea level in the Canneto valley. It does not consist of an aquifer and its source is supplied by the water catchment area of the river Melfa.

> Output: Flow_Rate

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_water_spring_madonna_di_canneto = pd.read_csv(get_path('madonna'))
msno.matrix(df_water_spring_madonna_di_canneto);

In [ ]:
df_water_spring_madonna_di_canneto.plot(subplots=True, figsize=(30, 13));

The following code is commented out because it stops the remainder of the notebook from running if choosing to "Run All" - however, it leads us to find some missing dates in the dataset.

In [ ]:
# commented out because it stops the entire notebook from running if choosing to "Run All"
# df_water_spring_madonna_di_canneto['Date'] = pd.to_datetime(df_water_spring_madonna_di_canneto['Date'], format = '%d/%m/%Y')
# missing_dates = determine_time_gaps(df_water_spring_madonna_di_canneto['Date'])
# print('Number of missing dates is {}'.format(len(missing_dates)))
# if len(missing_dates) > 0:
#     print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
#     print(pd.to_datetime(missing_dates.values))
# print('End of mostly missing data is {}'.format(df_water_spring_madonna_di_canneto.dropna()['Date'].values[0]))

It looks likes there are 9 missing dates at the end, where all the data is NaN as well. Let's remove those dates and check again.

In [ ]:
df_water_spring_madonna_di_canneto = pd.read_csv(get_path('madonna'))
df_water_spring_madonna_di_canneto = df_water_spring_madonna_di_canneto.iloc[:-9, :]
df_water_spring_madonna_di_canneto['Date'] = pd.to_datetime(df_water_spring_madonna_di_canneto['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps(df_water_spring_madonna_di_canneto['Date'])
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_water_spring_madonna_di_canneto.dropna()['Date'].values[0]))

There are no missing dates now. Let's filter for the target variable, where we have substantial data starting 2016-04-13, and for the independent variables, where we have data until 2018-12-31. 

In [ ]:
df_water_spring_madonna_di_canneto = df_water_spring_madonna_di_canneto[
    (df_water_spring_madonna_di_canneto['Date'] >= '2016-04-13') & (df_water_spring_madonna_di_canneto['Date'] <= '2018-12-31')]

In [ ]:
df_water_spring_madonna_di_canneto.plot(x='Date', subplots=True, figsize=(30, 13));

This is a bit of an unsual dataset. There are some missing values interspered even with this filtered data. Ideally, we would use the data from 2016-04-13 to 2018-02-03 as our training data, and 2018-08-23 to 2018-12-08 as our test data. However, the LSTM code is hard to modify for non-continuous datasets, so we will use the dataset as it is after interpolating, only using the last portion as our test set.

In [ ]:
df_water_spring_madonna_di_canneto.interpolate(method='linear', inplace=True)
df_water_spring_madonna_di_canneto.plot(x='Date', subplots=True, figsize=(30, 13));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_water_spring_madonna_di_canneto) - len(df_water_spring_madonna_di_canneto.dropna()))

## Lupa Water Spring

According to the provided description, 

> Description: this water spring is located in the Rosciano Valley, on the left side of the Nera river. The waters emerge at an altitude of about 375 meters above sea level through a long draining tunnel that crosses, in its final section, lithotypes and essentially calcareous rocks. It provides drinking water to the city of Terni and the towns around it.

> Output: Flow_Rate

First, let's take a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_water_spring_lupa = pd.read_csv(get_path('lupa'))
# df_water_spring_lupa['Date'] = pd.to_datetime(df_water_spring_lupa['Date'], format = '%d/%m/%Y')
msno.matrix(df_water_spring_lupa);

In [ ]:
df_water_spring_lupa['Date'] = pd.to_datetime(df_water_spring_lupa['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps(df_water_spring_lupa['Date'])
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_water_spring_lupa.dropna()['Date'].values[0]))

There are no missing dates. We can see that there is a large number of missing values early in the dataset, and then very sporadically after. We can drop the earliest rows and still have plenty of data with which to work. Upon visual inspection, most of the missing data ends on 2010-09-04, row 610. We will drop everything before this.

In [ ]:
df_water_spring_lupa = df_water_spring_lupa[df_water_spring_lupa['Date'] >= '2010-09-04']

In [ ]:
df_water_spring_lupa.plot(x='Date', subplots=True, figsize=(30, 10));

It looks like we can safely interpolate the few missing values in the dataset. However, in this particular case, we do not have enough variability in the data to use the entire last year as our test set. The flow rate decreased linearly between late 2010 - early 2020, and then a lot of rainfall in early 2020 substantially increased the flow rate. So instead of using the last year as our test set, we will use some of the last months instead.

In [ ]:
df_water_spring_lupa = df_water_spring_lupa.interpolate(method='linear')
print('Number of rows which still contain at least one missing value: ', len(df_water_spring_lupa) - len(df_water_spring_lupa.dropna()))

## Arno River

According to the provided description, 

> Description: Information about Arno river. The Arno is the second largest river in peninsular Italy and the main waterway in Tuscany and it  has a relatively torrential regime, due to the nature of the surrounding soils (marl and impermeable clays)

> Outut: Hydrometry_Nave_di_Rosano

First, let's take a look at the data:


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_river_arno = pd.read_csv(get_path('arno'))
msno.matrix(df_river_arno);

There are three possible strategies for handling the missing data. The first is to only use the columns where all the variables are available; the second is to only use the rows where all the variables are available; the third is to impute missing values. 

First, we must determine whether there are any missing dates. 

In [ ]:
df_river_arno['Date'] = pd.to_datetime(df_river_arno['Date'], format = '%d/%m/%Y')
missing_dates = determine_time_gaps((df_river_arno['Date']))
print('Number of missing dates is {}'.format(len(missing_dates)))
if len(missing_dates) > 0:
    print('Start of missing dates is {}, and end of missing dates is {}.'.format(missing_dates.values[0], missing_dates.values[-1]))
    print(pd.to_datetime(missing_dates.values))
print('End of mostly missing data is {}'.format(df_river_arno.dropna()['Date'].values[0]))

For the Arno River, there are no missing dates. We can drop all of the rows where most of the data is missing, which is all the data prior to 2004-01-01.

In [ ]:
df_river_arno = df_river_arno[df_river_arno['Date'] >= '2004-01-01']

Now we can look at the missing data again.

In [ ]:
msno.matrix(df_river_arno);

In [ ]:
df_river_arno.plot(x='Date', subplots=True, figsize=(30, 60));

There are several years of nearly continous data for all the variables. We can say that after extensive experimentation, the best results were achieved when training on these several years. Furthermore, the results were still not so bad when we imputed the missing values and then trained and predicted based on imputed values.

As before, we will fill in the missing values of the temperature variable with those of prior years. The final date for which we have temperature data is 2017-08-03.

We will then create two dataframes. One of them will have only the almost complete data from 2004-01-01 to 2007-07-06. The other will have imputed values for all of the missing values, and will be used in the last section of this notebook, where we demonstrate how to make predictions. 

There is potentially anomalous data for the target variable `Hydrometry_Nave_di_Rosano` for about half of 2008. However, this data will not be in our first dataset at all, and is not so relevant for our second dataset, which we are only using to demonstrate how to make future predictions, so we will leave it as is.

In [ ]:
df_river_arno.loc[(df_river_arno['Date'] >= '2017-08-04') & (df_river_arno['Date'] <= '2020-06-30'), 'Temperature_Firenze'] = \
    df_river_arno.loc[(df_river_arno['Date'] >= '2014-08-04') & (df_river_arno['Date'] <= '2017-06-30'), 'Temperature_Firenze'].values

In [ ]:
df_river_arno_full_data = df_river_arno.loc[(df_river_arno['Date'] >= '2004-01-01') & (df_river_arno['Date'] <= '2007-07-06')]
df_river_arno_full_data = fill_in_missing_data(df_river_arno_full_data, 'kNN')
df_river_arno_imputed_data = fill_in_missing_data(df_river_arno.copy(), 'kNN')

Let's take a look at our final two datasets. 

Dataset using only rows with all the variables available:

In [ ]:
df_river_arno_full_data.plot(x='Date', subplots=True, figsize=(30, 50));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_river_arno_full_data) - len(df_river_arno_full_data.dropna()))

Dataset using imputed data:

In [ ]:
df_river_arno_imputed_data.plot(x='Date', subplots=True, figsize=(30, 50));

In [ ]:
print('Number of rows which still contain at least one missing value: ', len(df_river_arno_imputed_data) - len(df_river_arno_imputed_data.dropna()))

# Feature Engineering

Before we begin making predictions, we must make it clear what exactly we are predicting. Let's use the Arno River as an example dataset. To keep things simple, we will only keep one of the rainfall variables, as well as the date, the temperature variable, and the target variable.

In [ ]:
example_df_river_arno = pd.read_csv(get_path('arno'))
example_df_river_arno = example_df_river_arno.dropna()[['Date', 'Rainfall_Le_Croci', 'Temperature_Firenze', 'Hydrometry_Nave_di_Rosano']]
example_df_river_arno = example_df_river_arno[80:]
example_df_river_arno.head()

In typical machine learning problems, we would be given the date, rainfall, and temperature variables, and try to predict the hydrometry. However, this is a time-series prediction problem. In our problem, on any given day, we have all of the data in the row available. What we want to do is to predict the target variables of the _following_ days. For this, we need to add the values of the following days' target variable to the dataframe. 

In [ ]:
def add_following_n_target_days(df, num_days_to_predict, name_of_target, diff):
    """
    Appends the values of the target on the following n days to every row

    Args:
        df - dataframe to which following argument days are being appended
        num_days_to_predict - numbers of target variables to be added for prediction
        name_of_target - name of the target variable (in case of trying to predict the difference between days)
        diff - boolean; if True, trying to compute the difference between days
    
    Returns:
        df - dataframe which has n target days appended
    """
    targets = []
    if not diff:
        for i in range(1, num_days_to_predict + 1):
            lookback_vars = df[name_of_target].shift(periods=-i, fill_value=np.nan)
            lookback_vars.name = lookback_vars.name + '_{}_days_forward'.format(i)
            targets.append('{}_{}_days_forward'.format(name_of_target, i))
            df = pd.merge(df, lookback_vars, how="left", left_index=True, right_index=True)
    else:
        for i in range(-10, num_days_to_predict):
            lookback_vars = df[name_of_target].shift(periods=-i, fill_value=np.nan)
            lookback_vars.name = lookback_vars.name + '_{}_days_forward'.format(i + 1)
            if i > -1:
                targets.append('{}_{}_days_forward'.format(name_of_target, i + 1))
            df = pd.merge(df, lookback_vars, how="left", left_index=True, right_index=True)  
        df.drop(columns=[name_of_target], inplace=True)
    return df, targets

In [ ]:
num_days_to_predict = 5
name_of_target = 'Hydrometry_Nave_di_Rosano'
diff = False
example_df_river_arno_with_n_targets_added, example_targets = add_following_n_target_days(
    example_df_river_arno, num_days_to_predict, name_of_target, diff)

In [ ]:
display(example_df_river_arno_with_n_targets_added.head())
print('The targets are: ', example_targets)

As we can see, we have each of the following 5 days' target variables added to each row. Logically, the values along the diagonals of these variables are the same.

It is important to remember that `Hydrometry_Nave_di_Rosano` is _not_ our target variable, it is part of our independent variables. We have access to the current day's hydrometry, as well as rainfall and temperature data, in order to make predictions for the following days. The target variables are `Hydrometry_Nave_di_Rosano_1_days_forward` ... `Hydrometry_Nave_di_Rosano_5_days_forward`. 

In reality, we will be trying to predict the following 30 days, for every day of our test set. If our test set is a year of data, we will be making 30 * 365 = 10,950 predictions. However, the LSTM model only sees this as 365 predictions, since it will predict a sequence of the next 30 days for every one of the 365 days in the test set. The amount of days to predict is very easy to change, it only requires changing one variable as the input to a function. 

It is also important to explain the feature extraction process. We want to extract as much information from the data as possible. For this reason, we will make extensive use of dummy variables. A dummy variable is used to encode categorical data for machine learning predictions. For example, a dummy variable could be answering the question of whether the temperature is above 25 degrees, with a 1 for yes and a 0 for no. Another example would be a set of dummy variables for every month. This would require 12 new columns, each one representing a month, with the value of that column being 1 if the datapoint is in that month, and 0 if it is not. The purpose of these dummy variables is explained in the introduction, and the process for creating them is copied below:

> In order to capture the effects of the non-linearities, the following feature extraction is proposed: we make dummy variables for every week and month of the year, dummy variables for temperature brackets, and then make cross terms between 1) the time dummy variables and the temperature dummy variables; 2) the time dummy variables and other variables; 3) the temperature dummy variables and other variables. 

The extraction of the dummy variables takes place in `create_temp_and_time_dummies_plus_cross_terms()`, which is called by `feature_engineering()`. The addition of these dummy variables results in dataframes that typically have thousands of features, as opposed to the maximum of 26 in the Auser Aquifer dataset (excluding the date). When we use XGBRegressor to determine the most important features, we add the data from prior rows - for example, we add the data from the prior thirty days - to every row, thus resulting in dataframes that typically have tens of thousands of features. It is this process that allows us to determine which independent variables, from which time periods and temperature brackets, affect which target variables, thus capturing the inherent non-linear dynamics present in the natural environment.

Additionally, it is important to understand the output we will use from XGBRegressor. XGBRegressor is not multi-output compatible - one must use a wrapper, or compute every target variable separately, in order to make multiple predictions. For this reason, we are only able to extract the most important features for predicting a specific day in advance - e.g., predicting the target variable 7 days in advance. Furthermore, because of computational constraints that Kaggle has (there are some problems using the GPU for XGBRegressor, and there are only two CPU cores allocated when using the GPU), we will only present the results from predicting one day. 

Also relevant is the fact that the most important features will vary depending on the time of year. Since we are usually using multiple years of data in our training set, that means that we will be seeing all of the most important features, which might obscure more nuanced observations.

We will see that XGBRegressor typically finds hundreds of important features out of the tens of thousands it has available. Ideally, we would combine all of the feature importances for all of the days we want to predict, and then simplify the results. Unfortunately, doing all of these computations is not feasible on Kaggle. However, when we tested our XGBRegressor model on a 20 core / 40 thread machine, no call to the `fit()` method ever took longer than a minute, suggesting that this is perfectly feasible with relatively modest resources.   

## Feature Engineering Code

In [ ]:
def scale_values(df, feature_range=(0, 1)):
    """
    Scale values as part of data preparation
    
    Args:
        df - the dataframe to be scaled
        feature_range - the range of values to which the features are scaled 

    Returns:
        df - scaled dataframe
        scaler - the scaler which scaled the dataframe
    """
    
    scaler = MinMaxScaler(feature_range)
    df = scaler.fit_transform(df)
    return df, scaler

def invert_difference(df, original_target, diff_values, rolling_sum=True):
    """
    Invert differences data to recover the actual predictions;
    used in conjunction with difference_between_days()
    
    Args:
        df - the original dataframe
        original_target - name of the original target variable (before differencing)
        diff_values - dataframe containing the values to be inverted
        rolling_sum - whether to take the difference relative to the original target values, or the day prior;
            e.g. if rolling_sum == True, then the inverted values will be i + 1, i + 1 + 2, i + 1 + 2 + 3, ...
                 if rolling_sum == False, then the inverted values will be i + 1, i + 2, i + 3, ...
            given how the differences were taken, rolling_sum should always be set to True, 
            but sometimes gives better results when set to False

    Returns:
        result - the inverted values
    """

    original_target_values = df[original_target]
    result = pd.merge(original_target_values, diff_values, how="right", left_index=True, right_index=True)
    for num_column, column_name in enumerate(result.columns):
        if num_column == 0:
            continue
        else:
            if rolling_sum:
                result[column_name] = result[column_name] + result.iloc[:, num_column - 1]
            else:
                result[column_name] = result[column_name] + original_target_values
    result.drop(columns=original_target, inplace=True)
    
    return result

def create_synthetic_data(df, num_time_blocks_to_add):
    """
    Creates synthetic data for training purposes
    currently, this is only used for the Arno River
    
    Args:
        df - dataframe which contains the original data
        num_time_blocks_to_add    
    
    Returns:
        df - dataframe with the differences between days appended
        new_name_of_target - name of the new (differenced) target to be predicted 
    """
 
    data_to_copy = df.iloc[:365*2, :]
    temp_df = pd.DataFrame(columns=df.columns)
    
    for i in range(num_time_blocks_to_add):
        rainfall_multiplicative_factor = np.random.uniform(0.5, 1.5)
        target_variable_multiplicative_factor = rainfall_multiplicative_factor * np.random.uniform(0.95, 1.05)
        temperature_multiplicative_factor = np.random.uniform(0.98, 1.02)
        random_num_days_to_shift = np.random.randint(-7, 7)
        data_to_copy = data_to_copy.shift(random_num_days_to_shift, fill_value=np.nan)
        data_to_copy[data_to_copy.columns.drop(['Date', 'Temperature_Firenze_synthetic', 'Hydrometry_Nave_di_Rosano_synthetic'])] = \
            data_to_copy[data_to_copy.columns.drop(['Date', 'Temperature_Firenze_synthetic', 'Hydrometry_Nave_di_Rosano_synthetic'])] * rainfall_multiplicative_factor
        data_to_copy['Temperature_Firenze_synthetic'] = data_to_copy['Temperature_Firenze_synthetic'] * temperature_multiplicative_factor
        data_to_copy['Hydrometry_Nave_di_Rosano_synthetic'] = data_to_copy['Hydrometry_Nave_di_Rosano_synthetic'] * target_variable_multiplicative_factor    
        temp_df = temp_df.append(data_to_copy)
    
    end_date = pd.to_datetime(df['Date'].values[0])    
    total_length_added = temp_df.shape[0]
    actual_end_date = end_date - timedelta(days=1)
    start_date = actual_end_date - timedelta(days=total_length_added - 1)
    dates_added = pd.Series(data=pd.date_range(start=start_date, end=actual_end_date, freq='D'))
    temp_df['Date'] = dates_added.values
    temp_df.append(df)['Date'].reset_index().drop(columns='index')
    temp_df = temp_df.append(df)
    temp_df = temp_df.reset_index().drop(columns='index')
    temp_df[temp_df.columns.drop('Date')] = KNN(k=10).fit_transform(temp_df[temp_df.columns.drop('Date')])
    return temp_df


def difference_between_days(df, name_of_target, num_days_difference=1):
    """
    Make the target variable be the difference between one day and the following day, instead of the raw value;
    used for non-stationary data
    
    Args:
        df - dataframe which is being altered
        name_of_target - target variable which we are differencing
        num_days_difference - number of following days to be from which we are taking the difference; 
                              should be == 1
    
    Returns:
        df - dataframe with the differences between days appended
        new_name_of_target - name of the new (differenced) target to be predicted 

    """

    new_name_of_target = name_of_target + '_diff'
    df[new_name_of_target] = df[name_of_target].shift(-num_days_difference)\
                                         - df[name_of_target] 
    return df, new_name_of_target

def create_temp_and_time_dummies_plus_cross_terms(df, name_of_target, name_of_temperature_variable, no_temp):
    """
    Feature extraction: make dummy variables for temperature brackets, periods of the year (weeks and month),
    and create cross terms between these dummy variables and the rainfall variables
    
    Args:
        df - dataframe to be amended 
        name_of_target - name of the target variable
        name_of_temperature_variable - name of the temperature variable; in case there is more than one, 
            pick one arbitrarily
        no_temp - boolean; True if there is no temperature variable
    
    Returns:
        df - dataframe with dummies variables and cross terms appended

    """
    
    temp_dummies = None
    if not no_temp:
        temp_dummies_dict = {}
        for temp_range in [[-100, 10], [10, 20], [20, 25], [25, 30], [30, 35], [35, 100]]:
            temp_dummies_dict['temp_group_{}-{}'.format(temp_range[0], temp_range[1])] = ((temp_range[0] < df[name_of_temperature_variable]) 
                                                                & (df[name_of_temperature_variable] <= temp_range[1])).astype(int)
        temp_dummies = pd.DataFrame(temp_dummies_dict)

    df['week_of_year'] = pd.DatetimeIndex(df['Date']).weekofyear
    # error keeps on popping up, tried many combinations, but couldn't find solution that worked properly
#     df['week_of_year'] = pd.DatetimeIndex(df['Date']).dt.isocalendar().week
#     df['week_of_year'] = pd.Int64Index(df['Date']).dt.isocalendar().week

    time_dummies = pd.get_dummies(df['week_of_year'], prefix='Week', drop_first=True)
    df['month_of_year'] = pd.DatetimeIndex(df['Date']).month
    month_time_dummies = pd.get_dummies(df['month_of_year'], prefix='Month', drop_first=True)
    time_dummies = time_dummies.join(month_time_dummies)

    if not no_temp:
        variables = df.columns.drop(['Date', name_of_temperature_variable])
    else:
        variables = df.columns.drop(['Date'])

    if not no_temp:
        for temp_dummy in temp_dummies.columns:
            for relevant_var in variables:
                df['{}_x_{}'.format(temp_dummy, relevant_var)] = np.multiply(temp_dummies[temp_dummy], df[relevant_var])

        for temp_dummy in temp_dummies.columns:
            for time_dummy in time_dummies:
                df['{}_x_{}'.format(temp_dummy, time_dummy)] = np.multiply(temp_dummies[temp_dummy], time_dummies[time_dummy])

    for time_dummy in time_dummies:
        for relevant_var in variables:
            df['{}_x_{}'.format(time_dummy, relevant_var)] = np.multiply(time_dummies[time_dummy], df[relevant_var])
        
    
    """these cross terms don't seem to improve predictions, but the code is left here just in case"""
#     for temp_dummy in temp_dummies.columns:
#         for time_dummy in time_dummies:
#             for relevant_var in rainfall_vars:
#                 df['{}_x_{}_x_{}'.format(temp_dummy, time_dummy, relevant_var)] = \
#                 np.multiply(np.multiply(temp_dummies[temp_dummy], time_dummies[time_dummy]), df[relevant_var])

    df = time_dummies.join(df)
    if not no_temp:
        df = temp_dummies.join(df)
        
    del [temp_dummies, time_dummies, month_time_dummies]
    gc.collect()
    
    return df


def add_lookback_window(df, num_lookback_days):
    """
    Appends num_lookback_timesteps number of prior days' data to a given row.
    i.e. if num_lookback_timesteps == 1, we will append to every row the data in the prior row;
         if num_lookback_timesteps == 2, we will append to every row the data in the prior two rows
         
    Args:
        df - dataframe to which lookback days are being appended
        num_lookback_timesteps - number of prior timesteps days which are being added
    
    Returns:
        df - dataframe with lookback days appended
        lookback_dict - a dictionary mapping a digit i to the variables from i days prior 
    """
        
    temp_df = df.copy()
    lookback_dict = {}
    for i in range(1, num_lookback_days + 1):
        
        libc.malloc_trim(0)
        
        lookback_vars = temp_df.shift(periods=i, fill_value=np.nan)
        lookback_vars = lookback_vars.add_suffix('_{}_days_prior'.format(i))
        lookback_dict[i] = list(lookback_vars.columns)
        df = pd.merge(lookback_vars, df, how="left", left_index=True, right_index=True)        
    
    libc.malloc_trim(0)
    del [temp_df]
    gc.collect()
    
    return df, lookback_dict


def add_following_n_target_days(df, num_days_to_predict, name_of_target, diff):
    """
    Appends the values of the target on the following num_days_to_predict days to every row
    
    Args:
        df - dataframe to which following argument days are being appended
        num_days_to_predict - numbers of target variables to be added for prediction
        name_of_target - name of the target variable (in case of trying to predict the difference between days)
        diff - boolean; if True, trying to compute the difference between days
    
    Returns:
        df - dataframe which has n target days appended
        targets - names of the new target variables
        target_dict - a dictionary mapping a digit i to the corresponding target variable i days in the future
    """
    
    targets = []
    target_dict = {}
    if not diff:
        for i in range(1, num_days_to_predict + 1):

            libc.malloc_trim(0)
            
            lookforward_vars = df[name_of_target].shift(periods=-i, fill_value=np.nan)
            lookforward_vars.name = lookforward_vars.name + '_{}_days_forward'.format(i)
            targets.append('{}_{}_days_forward'.format(name_of_target, i))
            target_dict[i] = targets[-1]
            df = pd.merge(df, lookforward_vars, how="left", left_index=True, right_index=True)
    else:
        for i in range(-1, num_days_to_predict):
            
            libc.malloc_trim(0)
            
            lookforward_vars = df[name_of_target].shift(periods=-i, fill_value=np.nan)
            lookforward_vars.name = lookforward_vars.name + '_{}_days_forward'.format(i + 1)
            if i > -1:
                targets.append('{}_{}_days_forward'.format(name_of_target, i + 1))
            target_dict[i] = targets[-1]
            df = pd.merge(df, lookforward_vars, how="left", left_index=True, right_index=True)  
            
        df.drop(columns=[name_of_target], inplace=True)
        
    libc.malloc_trim(0)
    gc.collect()
        
    return df, targets, target_dict


def feature_engineering(df, name_of_target, name_of_temp_variable, num_lookback_days, num_days_to_predict, 
                        num_days_difference, synthetic, print_flags, no_temp=False):
    """
    This function will create take the dataset and prepare it for analysis. Depending on the parameters,
    it will do the following in order:
        1) create synthetic data for training on additional date (currently only done for the Arno River)
        2) make the target variable be the difference between days, instead of the raw value
        3) create temperature and time dummy variables, as well as cross terms between them
        4) append num_lookback_days number of prior days' data to a given row
        5) append the values of the target on the following num_days_to_predict days to every row

    Args:
        df - dataframe containing the raw values
        name_of_target - name of the target variable
        name_of_temp_variable - name of the temperature variable
        num_lookback_days - number of prior days' data to append to row; if 0, won't append
        num_days_to_predict - number of days in advance the target variable is being predicted
        num_days_difference - make the target variable be the difference between one day and num_days_difference 
                              following days; if 0, will use original values
        synthetic - create synthetic data on which to train; if 0, none will be made
                    currently only for use on the Arno River
        print_flags - if True, will print information about intermediate dataframes and feature engineering
        no_temp - boolean; True if there is no temperature variable
    
    Returns:
        df - original dataframe plus dummy variables and cross terms 
        targets - the target variables
        original_target_values - the column containing the original target values, needed for invert_difference()
        lookback_dict - a dictionary mapping a digit i to the variables from i days prior
        target_dict - a dictionary mapping a digit i to the corresponding target variable i days in the future
    """

    
    # this was already done during the data cleaning process, but is left here in case of new data
    df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')
        
    if synthetic:
        if print_flags:
            print('starting data:')
            display(df)
            print('creating synthetic data')
        df = create_synthetic_data(df, synthetic)
        if print_flags:
            print('done with synthetic data')
            display(df)
    
    if num_days_difference:
        if print_flags:
            print('creating differences between days')
        df, new_name_of_target = difference_between_days(df, name_of_target, num_days_difference)
        if print_flags:
            print('done with creating differences between days')        
            display(df)
    else:
        new_name_of_target = name_of_target
    
    if print_flags:
        print('creating temperature and time dummy variables plus cross terms')

    
    df = create_temp_and_time_dummies_plus_cross_terms(df, name_of_target, name_of_temp_variable, no_temp)    
    df.set_index('Date', inplace=True)
    
    if print_flags:
        print('done with creating temperature and time dummy variables plus cross terms')
        display(df)
    
    lookback_dict = None
    if num_lookback_days:
        if print_flags:
            print('creating lookback days')
        df, lookback_dict = add_lookback_window(df, num_lookback_days)
        if print_flags:
            print('done with creating lookback days')    
            display(df)

    if print_flags:
        print('creating target variables for following days')
        
    # send target variable to last column
    libc.malloc_trim(0)
    original_target_values = df.loc[:, name_of_target]
    libc.malloc_trim(0)
    df = df.drop(columns=name_of_target)
    gc.collect()
    df[name_of_target] = original_target_values
    
    # send target variable to last column in case target variable is now the difference between days
    # if it isn't, we are just repeating the step above, with no consequences
    temporary_target_data = df.loc[:, new_name_of_target]
        
    libc.malloc_trim(0)
    df = df.drop(columns=new_name_of_target)
    gc.collect()

    df[new_name_of_target] = temporary_target_data        
    df, targets, target_dict = add_following_n_target_days(df, num_days_to_predict, new_name_of_target, num_days_difference)
    
    libc.malloc_trim(0)
    df = df.dropna()
    gc.collect()
        
    if print_flags:
        print('done with creating target variables for following days')
        print('final dataset: ')
        display(df)
    
    del [temporary_target_data]
    gc.collect()
    
    libc.malloc_trim(0)
    
    return df, targets, original_target_values, lookback_dict, target_dict

# Prediction Code

In this section, we present the code for making predictions. The functions can take many different parameters, making predictions quite flexible.  

## Train-test-split Code

In [ ]:
def make_training_and_test_sets(df, targets, test_size, lookback_period, num_days_to_predict):
    """
    Splits the data into training and test sets; we must be careful to avoid leaking test data when we train
    
    The split is done as follows: if we want to make test_size predictions on our test set, 
        we need to make our training set size equal to test_size + lookback_period, since the TimeseriesGenerator()
        function used to format data for the LSTM will use lookback_period number of days as starting input for
        day lookback_period + 1; effectively, the first lookback_period datapoints are not predicted, 
        leaving us with test_size predictions
    We don't want to leak any test data into our model, so we leave a gap of size num_days_to_predict between
        our training and test dates, since the last row of the training data will have information for the next
        num_days_to_predict target variables
        
    Args:
        df - dataframe containing the data to split
        targets - the target variables
        test_size - the number of datapoints in our test set
        lookback_period - the number of prior days that LSTM will be looking at when making predictions
        num_days_to_predict - number of days to be predicting; in order to avoid leaking data into our model, the 
                              training set is cut off num_days_to_predict days prior to the size of the test set
    
    Returns:
        X_train, y_train - training data
        X_test, y_test - testing data
    """
        
    training_set = df.iloc[:-test_size - lookback_period - num_days_to_predict, :]
    test_set = df.iloc[-test_size - lookback_period:, :]
    X_train = training_set.iloc[:, :-num_days_to_predict]
    y_train = training_set[targets]
    X_test = test_set.iloc[:, :-num_days_to_predict]
    y_test = test_set[targets]
    
    return X_train, y_train, X_test, y_test

## XGBRegressor Code

In [ ]:
def run_xgb(df, targets, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict):
    """
    Run XGBRegressor on the dataset and saves the model
    
    Args:
        df - dataframe containing the data
        targets - names of target variables, i.e. Hydrometry_Nave_di_Rosano_7_days_forward
        test_size - size of the test set
        num_lookback_days - number of days of prior data included in any given row
        num_days_to_predict - number of days to predict
    
    Returns:
        xgb_feature_importances_path - path to file containing the importances of the features as 
                                       determined by XGBRegressor 
        features_path - path to file containing the features trained on in the model
        y_test_path - path to file containing the test values
        y_pred_path - path fo file containing the predictions
        
    """
    
# apparently, there is a known bug when using XGB on GPUs where the memory leaks, so for now just use CPU
# in Kaggle
#     if not kaggle_path:
#         params = {'n_estimators': 100,
#               'max_depth': 4,
#               'subsample': 0.7,
#               'learning_rate': 0.04,
#               'random_state': 0}    
#     else:
#         params = {'tree_method': 'gpu_hist',
#               'n_estimators': 100,
#               'max_depth': 4,
#               'subsample': 0.7,
#               'learning_rate': 0.04,
#               'random_state': 0,
#               'n_gpus': 1}

    params = {'n_estimators': 100,
              'max_depth': 4,
              'subsample': 0.7,
              'learning_rate': 0.04,
              'random_state': 0}    

    
    xgb_regressor = xgb.XGBRegressor(**params)

    X_train, y_train, X_test, y_test = \
                        make_training_and_test_sets(df, targets, test_size, num_lookback_days, num_days_to_predict)

    if not os.path.exists(get_path('xgb')):
        os.makedirs(get_path('xgb'))
    if not os.path.exists(get_path('xgb_results')):
        os.makedirs(get_path('xgb_results'))
        
    current_target = y_test.columns[which_day_to_predict - 1]
    y_train = y_train.iloc[:, which_day_to_predict - 1]
    y_test = y_test.iloc[:, which_day_to_predict - 1]
    xgb_model = xgb_regressor.fit(X_train, y_train)      
    
    libc.malloc_trim(0)
    gc.collect()
    libc.malloc_trim(0)
    
    y_pred = xgb_model.predict(X_test)
    
    libc.malloc_trim(0)
    gc.collect()
    
    xgb_feature_importances = xgb_model.feature_importances_
    features = X_train.columns
    
    xgb_feature_importances_path = '{}/{}_feature_importances'.format(get_path('xgb_results'), current_target)
    features_path = '{}/{}_features'.format(get_path('xgb_results'), current_target)   
    y_test_path = '{}/{}_actual'.format(get_path('xgb_results'), current_target)
    y_pred_path = '{}/{}_predictions'.format(get_path('xgb_results'), current_target)
    
    pd.Series(xgb_feature_importances).to_csv(xgb_feature_importances_path)    
    pd.Series(features).to_csv(features_path)    
    pd.Series(y_test).to_csv(y_test_path)    
    pd.Series(y_pred).to_csv(y_pred_path)
    
    xgb_model.save_model('{}/{}'.format(get_path('xgb'), current_target))
    
    del [xgb_model, xgb_regressor]
    
    libc.malloc_trim(0)
    gc.collect()
    libc.malloc_trim(0)
    
    
    libc.malloc_trim(0)
    del [X_train, y_train, X_test, xgb_feature_importances, features, y_test, y_pred]
    gc.collect()
    
    return xgb_feature_importances_path, features_path, y_test_path, y_pred_path


def get_n_most_important_feature_indices(feature_importances_path, features_path, name_of_target, num_most_important_features):
    """
    Retrieve and print information about the most important features determined by XGBRegressor
    
    Args:
        xgb_feature_importances_path - path to file containing the importances of the features as 
                                       determined by XGBRegressor 
        features_path - path to file containing the features trained on in the model
        name_of_target - name of the target being predicted
        num_most_important_features - number of most important features (in sorted order) we want to pring 
    
    Returns:
        sorted_feature_importances_pairs - a list of pairs (feature, importance) in decreasing order of importance
    """
    
    feature_importances = pd.read_csv(feature_importances_path).iloc[:, 1]
    features = pd.read_csv(features_path).iloc[:, 1]
    
    feature_importances_indices_mask = np.where(feature_importances > 0, True, False)

    relevant_features = features[feature_importances_indices_mask]
    relevant_features_importances = feature_importances[feature_importances_indices_mask]
    
    feature_importances_pairs = list(zip(relevant_features, relevant_features_importances))
    sorted_feature_importances_pairs = sorted(feature_importances_pairs, key=lambda x: x[1], reverse=True)
    
    num_nonzero_important_features = len(sorted_feature_importances_pairs)
    
    print('Predicting {}'.format(name_of_target))
    print()
    print('Out of {} features, XGBRegressor found {} to be useful for prediction.'.format(len(features), 
                                                                        num_nonzero_important_features))
    print()
    print('The following are the first {} most important features, sorted in order of decreasing importance: '
          .format(min(num_nonzero_important_features, num_most_important_features)))
    print()
    for i in range(min(num_nonzero_important_features, num_most_important_features)):
        print('{}, {}'.format(sorted_feature_importances_pairs[i][0], sorted_feature_importances_pairs[i][1]))

## LSTM Code

In [ ]:
def fit_lstm(X_train, y_train, batch_size, num_lookback_days, num_epochs):    
    """
    Create an LSTM neural network and fit it on the training data
    
    Args:
        X_train - independent training variables
        y_train - target variables
        batch_size - batch size for the neural network
        num_lookback_days - number of prior days for the LSTM to look at
        num_epochs - number of training epochs

    Returns:
        model - the resulting LSTM neural network
    """ 
    
    n_input = num_lookback_days
    len_output = y_train.shape[1]
    n_features= X_train.shape[1] 
    generator = TimeseriesGenerator(X_train, y_train, length=num_lookback_days, batch_size=batch_size)

    # One of several alternative models:    
#     model = Sequential()
#     model.add(Conv1D(filters=64, kernel_size=3, activation='tanh', input_shape=(n_input,n_features)))
#     model.add(Conv1D(filters=64, kernel_size=3, activation='tanh'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Flatten())
#     model.add(RepeatVector(len_output))
#     model.add(LSTM(200, activation='tanh', return_sequences=True, unit_forget_bias=False))
#     model.add(TimeDistributed(Dense(100, activation='tanh')))
#     model.add(TimeDistributed(Dense(1)))
#     model.compile(loss='mae', optimizer='adam')
#     print(model.summary())
#     print(generator[0][0].shape)
#     model.fit(generator, epochs=num_epochs)

    # One of several alternative models:
#     model = Sequential()
#     model.add(LSTM(200, input_shape=(n_input, n_features), return_sequences=True, stateful=True,  batch_size=batch_size))
#     model.add(LeakyReLU(alpha=0.05))
#     model.add(LSTM(200, input_shape=(n_input, n_features), return_sequences=True, stateful=True,  batch_size=batch_size))
#     model.add(LeakyReLU(alpha=0.05))
#     model.add(Dense(len_output))
#     model.compile(optimizer='adam', loss='mae')
#     print(model.summary())
#     print(generator[0][0].shape)
#     print('hit')
#     model.fit(generator, epochs=num_epochs)
    
    # One of several alternative models:
    model = Sequential()
    model.add(LSTM(200, activation='tanh', input_shape=(n_input, n_features)))
    model.add(RepeatVector(len_output))
    model.add(LSTM(200, activation='tanh', return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation='tanh')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mae', optimizer='adam')
    print(model.summary())
    print(generator[0][0].shape)
    print('hit')
    model.fit(generator, epochs=num_epochs)

    
    
    libc.malloc_trim(0)
    gc.collect()
    
    return model

    
def train_prior_years(df, targets, size_test_set, num_lookback_days, num_days_to_predict, model, j,
                      num_train_years_prior, batch_size, num_epochs_train_prior_years,
                      X_scaler, y_scaler=None):
    """
    Train on data from prior years around the same time before making a prediction
    
    Arguments are originally passed from run_lstm(), see run_lstm() for details
    
    Returns:
        model - the updated LSTM neural network
    
    """
    
    # train on data from prior years within an interval one week before and one week after the current date
    for i in range(num_train_years_prior, 0, -1):
        libc.malloc_trim(0)

        X_train, y_train, X_test, y_test = make_training_and_test_sets(df, targets, i * 365 \
                                                        + size_test_set - j + 7 + num_days_to_predict, 
                                                            num_lookback_days, num_days_to_predict)
        X_test = X_test.iloc[:num_lookback_days + 2*7 + 1, :]
        y_test = y_test.iloc[:num_lookback_days + 2*7 + 1, :]
        if y_scaler:
            X_test_scaled = X_scaler.fit_transform(X_test)
            y_test_scaled = y_scaler.fit_transform(y_test)
            test_generator = TimeseriesGenerator(X_test_scaled, y_test_scaled, length=num_lookback_days, 
                                                 batch_size=batch_size)
        else: 
            test_generator = TimeseriesGenerator(X_test, y_test, length=num_lookback_days, 
                                     batch_size=batch_size)
        print('Training on data from {} years prior'.format(i))
        libc.malloc_trim(0)
        model.fit(test_generator, epochs=num_epochs_train_prior_years)
        libc.malloc_trim(0)
        print('Finished training on data from {} years prior'.format(i))
                
    return model

def lstm_walk_forward(df, targets, size_test_set, num_lookback_days, num_days_to_predict, model, 
                      num_train_years_prior, batch_size, num_epochs_walk_forward, num_epochs_train_prior_years,
                      y_test_actual, original_y_test, X_scaler, y_scaler=None):
    """
    Implementation of walk-forward validation. As predictions are being made, update the model on the most 
    recently available data. For example, if predictions are being made for 30 days in advance, 
    then we would update the model with the data from 30 days ago
    
    Arguments are originally passed from run_lstm(), se run_lstm() for details
    
    Returns:
        df_pred - dataframe containing the predictions; dimensions = (size_test_set, num_days_to_predict)
        model - the updated LSTM neural network
    """
    
    print('walking forward')
    y_preds = []
    
    for j in range(size_test_set):              
        # update model with data from num_days_to_predict prior (i.e., using information that would 
        # actually be available in a real-world situation)
        print('Iteration {} out of {}'.format(j, size_test_set))
        
        if j >= num_days_to_predict:
            X_train, y_train, X_test, y_test = make_training_and_test_sets(df, targets, 
                                               size_test_set - j + 2 * num_days_to_predict, 
                                                                    num_lookback_days, num_days_to_predict)
            X_test = X_test.iloc[:num_lookback_days + 1, :]
            y_test = y_test.iloc[:num_lookback_days + 1, :]

            if y_scaler:
                X_test_scaled = X_scaler.fit_transform(X_test)
                y_test_scaled = y_scaler.fit_transform(y_test)
                test_generator = TimeseriesGenerator(X_test_scaled, y_test_scaled, length=num_lookback_days, 
                                                     batch_size=batch_size)
            else: 
                test_generator = TimeseriesGenerator(X_test, y_test, length=num_lookback_days, 
                                     batch_size=batch_size)
            print('Training on data from {} days ago'.format(num_days_to_predict))
            model.fit(test_generator, epochs=num_epochs_walk_forward)
            print('Finished training on data from {} days ago'.format(num_days_to_predict))

            libc.malloc_trim(0)                        
            
        # train on data from prior years within an interval one week before and one week after the current date
        if num_train_years_prior:
            model = train_prior_years(df, targets, size_test_set, num_lookback_days, num_days_to_predict, model, j,
                      num_train_years_prior, batch_size, num_epochs_train_prior_years, X_scaler, y_scaler)

        # this is where we actually make the prediction for the next 30 days
        X_train, y_train, X_test, y_test = make_training_and_test_sets(df, targets, 
                                                                       size_test_set - j + num_days_to_predict, 
                                                                       num_lookback_days, num_days_to_predict)
        X_test = X_test.iloc[:num_lookback_days + 1, :]
        y_test = y_test.iloc[:num_lookback_days + 1, :]
        cur_day_index = y_test.index[-1]
        
        if y_scaler:
            X_test_scaled = X_scaler.fit_transform(X_test)
            y_test_scaled = y_scaler.fit_transform(y_test)
            test_generator = TimeseriesGenerator(X_test_scaled, y_test_scaled, length=num_lookback_days, 
                                                 batch_size=batch_size)
        else: 
            test_generator = TimeseriesGenerator(X_test, y_test, length=num_lookback_days, 
                                 batch_size=batch_size)

        y_pred_scaled = None
        if scale_y:
            y_pred_scaled = model.predict(test_generator)
            y_pred = y_scaler.inverse_transform(y_pred_scaled[:, :, 0])
        else:
            y_pred = model.predict(test_generator)[:, :, 0]
        
        y_preds.append(y_pred[0])
        
        print('Just completed predictions for ', y_test.index[-1])

        # if leak_data == True, then we will train on the data that we just predicted. Since the training
        # data includes the values of the target variables for the next 30 days, doing this will leak data
        # It serves as a good benchmark, but should default to False
        if leak_data:
            model.fit(test_generator, epochs=num_epochs)
            
        libc.malloc_trim(0)            
        del [test_generator, X_train, y_train, X_test, y_test, y_pred]
        if y_scaler:
            del [X_test_scaled, y_test_scaled, y_pred_scaled]
        gc.collect()
        libc.malloc_trim(0)

    df_pred = pd.DataFrame(data=y_preds, index=y_test_actual.index, columns=original_y_test.columns)
    return df_pred, model


def run_lstm(df, targets, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, original_target, 
             scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, 
             num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum):
    """
    df - dataframe containing raw values plus feature-engineered values
    targets - names of the target variables, i.e. Hydrometry_Nave_di_Rosano_7_days_forward
    num_lookback_days - number of prior days for the LSTM to look at
    num_days_to_predict - number of days to predict
    size_test_set - size of the test set
    num_days_difference - make the target variable be the difference between one day and num_days_difference 
                          following days; if 0, will use original values
    original_target - name of original target variable ()
    scale_y - boolean; if True, scale the target variable(s)
    walk_forward - boolean; if True, while predicting test data, train on most recently available data without
                            leaking
    leak_data - boolean; if True, while predicting test data, the model will train on the data it just predicted;
                            useful for benchmarking, but should be set to False
    synthetic - number of timeblocks of synthetic data to make for training; currently only compatible with 
                the Arno River
    num_train_years_prior - number of prior year intervals we train on before making a prediction
    batch_size - batch size
    num_epochs - number of epochs to train during the first fit
    num_epochs_walk_forward - if using walk-forward validation, the number of epochs to train on prior data
    num_epochs_train_prior_years - if training on data from prior year intervals, the number of epochs to train
                                   on each interval
    rolling_sum - boolean; if True, uses a rolling sum to invert differences - see invert_difference()
    """
        
    X_train, y_train, X_test, y_test = make_training_and_test_sets(df, targets, size_test_set, 
                                                                   num_lookback_days, num_days_to_predict)
    
    X_train_scaled, X_scaler = scale_values(X_train, feature_range=(0, 1))    
    X_test_scaled = X_scaler.fit_transform(X_test)
    original_y_test = y_test.copy()
    y_test_actual = y_test.iloc[-size_test_set:, :]
    df_pred = pd.DataFrame(index=y_test_actual.index, columns=original_y_test.columns)
    
    if not walk_forward:

        num_features= X_train.shape[1] 
        len_output = y_train.shape[1]        
        test_generator = TimeseriesGenerator(X_test_scaled, np.zeros(len(X_test)), length=num_lookback_days, batch_size=batch_size)
                
        if scale_y:
            feature_range=(0, 1) if not num_days_difference else (-1, 1)
            y_train_scaled, y_scaler = scale_values(y_train, feature_range)
            y_test_scaled = y_scaler.fit_transform(y_test)
            model = fit_lstm(X_train_scaled, y_train_scaled, batch_size, num_lookback_days, num_epochs)
            libc.malloc_trim(0)
            y_pred_scaled = model.predict(test_generator)
            libc.malloc_trim(0)            
            y_pred = y_scaler.inverse_transform(y_pred_scaled[:, :, 0])
    #         y_pred = y_scaler.inverse_transform(y_pred_scaled[:, -1, :])  # for other LSTM models
        else:    
            model = fit_lstm(X_train_scaled, y_train.to_numpy(), batch_size, num_lookback_days, num_epochs)    
            y_pred = model.predict(test_generator)
            y_pred = y_pred[:, :, 0]    
    #         y_pred = y_pred[:, -1, :]  # for other LSTM models
        df_pred = pd.DataFrame(y_pred, index=y_test_actual.index, columns=y_test_actual.columns)
    else:
        if scale_y:
            feature_range=(0, 1) if not num_days_difference else (-1, 1)
            y_train_scaled, y_scaler = scale_values(y_train, feature_range)
            model = fit_lstm(X_train_scaled, y_train_scaled, batch_size, num_lookback_days, num_epochs)
            df_pred, model = lstm_walk_forward(df, targets, size_test_set, num_lookback_days, num_days_to_predict, model, 
                      num_train_years_prior, batch_size, num_epochs_walk_forward, num_epochs_train_prior_years,
                                       y_test_actual, original_y_test, X_scaler, y_scaler)
        
        else:
            y_scaler = None
            model = fit_lstm(X_train_scaled, y_train.to_numpy(), batch_size, num_lookback_days, num_epochs)    
            df_pred, model = lstm_walk_forward(df, targets, size_test_set, num_lookback_days, num_days_to_predict, model, 
                      num_train_years_prior, batch_size, num_epochs_walk_forward, num_epochs_train_prior_years,
                                       y_test_actual, original_y_test, X_scaler, y_scaler)

    # if predicting the differences between days, invert the differences to retrieve the real values
    if num_days_difference: 
        y_test_actual = invert_difference(df, original_target, original_y_test.iloc[- size_test_set:, :])
        df_pred = invert_difference(df, original_target, df_pred, rolling_sum)
    
    del [df, X_train, y_train, X_test, y_test, X_train_scaled, X_test_scaled, original_y_test]
    gc.collect()

    if not os.path.exists(get_path('lstm')):
        os.makedirs(get_path('lstm')) 
    if not os.path.exists(get_path('lstm_results')):
        os.makedirs(get_path('lstm_results'))
        
    y_test_actual_path = '{}/{}_actual'.format(get_path('lstm_results'), original_target)
    df_pred_path = '{}/{}_predictions'.format(get_path('lstm_results'), original_target)
    y_test_actual.to_csv(y_test_actual_path)    
    df_pred.to_csv(df_pred_path)
    model.save('{}/LSTM_{}.h5'.format(get_path('lstm'), original_target))
    del model
    gc.collect()
    keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()
    
    model = None
    
    return df_pred_path, y_test_actual_path, model



## Plotting Results Code

In [ ]:
def plot_widget_results(df_pred_path, y_test_actual_path, name_of_target, dimension, num_days_in_advance=7):
    """
    Makes an interactive widget that displays the actual values, target values, the difference between them,
    mean absolute error, and root mean squared error
    
    Args:
        df_pred_path - path to predicted test values
        y_test_actual_path - ath to actual test values 
        name_of_target - name of the target variable
        dimension - type of variable; y-axis label
        num_days_in_advance - how many days in advance the prediction is; defaults to 7

    Returns:
        fig - the fig (although this parameter isn't strictly necessary)

    """
    
    %matplotlib notebook
    #calling it a second time may prevent some graphics errors
    %matplotlib notebook  
    import matplotlib.pyplot as plt

    def plot_predictions_vs_test_values(pred, y_test, name_of_target, dimension, num_days_in_advance):
        plt.clf()
        x = pd.to_datetime(y_test.index)
        y_test_values = y_test.iloc[:, num_days_in_advance - 1]
        y_predicted_values = pred.iloc[:, num_days_in_advance - 1]
        plt.plot(x, y_test_values, color='blue', alpha=0.7)
        plt.plot(x, y_predicted_values, color='red', alpha=0.7)
        plt.gca().fill_between(x, 
                               y_test_values, 
                               y_predicted_values, 
                               facecolor='purple', 
                               alpha=0.2)

        rmse = mean_squared_error(y_test_values, y_predicted_values, squared=False)
        mae = mean_absolute_error(y_test_values, y_predicted_values)
        rmse = np.round(rmse, 3)
        mae = np.round(mae, 3)

        fig = plt.gcf()
        if not kaggle_path:
            fig.set_size_inches(13.5, 9.5)
        ax = plt.gca()
        plt.legend(['actual', 'predicted', 'difference'], fontsize='large')
        plt.xlabel('Date', size=15)
        plt.ylabel(dimension, size=15)
        plt.title(name_of_target + ': LSTM', size=20)
        plt.show()

        ax.annotate('rmse: {}    mae: {}'.format(rmse, mae),
                xy=(0.17, -0.015), xytext=(0, 10),
                xycoords=('axes fraction', 'figure fraction'),
                textcoords='offset points',
                size=20, ha='center', va='bottom')

        return fig

    df_pred = pd.read_csv(df_pred_path)
    y_test_actual = pd.read_csv(y_test_actual_path)
    df_pred.set_index('Date', inplace=True)
    y_test_actual.set_index('Date', inplace=True)
    
    pd.set_option('display.max_columns', 30)
    diff = y_test_actual.subtract(df_pred)
    mae_all = np.mean(abs(diff), axis=0)
    rmse_all = np.sqrt(np.mean(abs(diff**2), axis=0))
    mae_all = np.round(mae_all, 3)
    rmse_all = np.round(rmse_all, 3)
    errors_all = pd.DataFrame(data=[mae_all, rmse_all], index=['MAE', 'RMSE'], columns=y_test_actual.columns)
    display(errors_all)
    
    plot_predictions_vs_test_values(df_pred, y_test_actual, name_of_target, dimension, num_days_in_advance=7)


    def update(change):
        plot_predictions_vs_test_values(df_pred, y_test_actual, name_of_target, dimension, change.new)

    int_slider = widgets.IntSlider(
        value=7, 
        min=1, max=30, step=1,
        description='num_days',
        continuous_update=False
    )
    int_slider.observe(update, 'value')
    int_slider
    
    return int_slider


def plot_xgb_results(df_pred_path, y_test_actual_path, name_of_target, dimension, num_days_in_advance):
    """
    Plot the actual vs. those predicted by XGBRegressor

    Args:
        df_pred_path - path to predicted test values
        y_test_actual_path - path to actual test values 
        name_of_target - name of the target variable
        dimension - type of variable; y-axis label
        num_days_in_advance - how many days in advance the prediction is; defaults to 7

    Returns:
        fig - the fig (although this parameter isn't strictly necessary)

    """
    
    %matplotlib notebook

    df_pred = pd.read_csv(df_pred_path)
    y_test_actual = pd.read_csv(y_test_actual_path)
    
    plt.clf()
    x = pd.to_datetime(y_test_actual.values[:, 0])
    y_test_values = y_test_actual.values[:, 1].astype('float64')
    y_predicted_values = df_pred.values[:, 1].astype('float64')
    plt.plot(x, y_test_values, color='orange', alpha=0.7)
    plt.plot(x, y_predicted_values, color='red', alpha=0.7)
    plt.gca().fill_between(x, 
                           y_test_values, 
                           y_predicted_values, 
                           facecolor='yellow', 
                           alpha=0.2)

    rmse = mean_squared_error(y_test_values, y_predicted_values, squared=False)
    mae = mean_absolute_error(y_test_values, y_predicted_values)
    rmse = np.round(rmse, 3)
    mae = np.round(mae, 3)
    
    fig = plt.gcf()
    fig.set_size_inches(13.5, 9.5)
    ax = plt.gca()
    plt.legend(['actual', 'predicted', 'difference'], fontsize='large')
    plt.xlabel('Date', size=20)
    plt.ylabel(dimension, size=20)
    plt.title(name_of_target + ' {} days ahead: XGBRegressor'.format(num_days_in_advance), size=20)
    plt.show()

    ax.annotate('rmse: {}    mae: {}'.format(rmse, mae),
            xy=(0.17, 0), xytext=(0, 30),
            xycoords=('axes fraction', 'figure fraction'),
            textcoords='offset points',
            size=20, ha='center', va='bottom')

    return ax

# Predictions

In this section, we use XGBRegressor to make predictions and determine which features are the most important for making predictions, and make predictions using LSTM. 

In many of the predictions, we will see the impact of the limitations which were part of the competition rules regarding making forecasts based on predicted values. In particular, for a number of predictions we will see a significant lag relative to the actual data. This is due to the fact that when we do our walk-forward validation, we can only update our current model with information from `num_days_to_predict` days ago, since that is when we have all of the real data necessary to train our model; if we trained it on more recent data, our model would have data leakage, meaning that our predictions would not reflect real-life performance. However, if we were allowed to train the model based on, for example, the predictions made yesterday for the following `num_days_to_predict` days (rather than the real values for the following `num_days_to_predict` days), these models would probably become much more accurate. However, this not was not allowed by the competition rules.

Nevertheless, most of the predictions are still quite accurate. It is also important to note that experimenting further with batch sizes and epochs, as well as other parameters like the number of layers, filters, and neurons in our LSTM model subtantially improves results, and much can be done to make these predictions even more accurate. The parameters here were chosen as a trade-off between performance and runtime. Additionally, once a model has been trained, one can make predictions with it immediately (as we demonstrate in the last section of this notebook), and it is very easy to update the model with new data. Thus, computational constraints should be viewed through the lens of having a large amount of computing power (as opposed to the computing power available on Kaggle), which only ever has to be used once to train the model, and which can then be deployed and updated quickly and easily afterwards. Multiple cloud providers can train these models for only a few dollars.

Additionally, since we can easily make predictions using XGBRegressor after training it to find the most important features, we will also display the predictions that XGBRegressor makes. We will see that it is often the case that when LSTM underestimates, XGBRegressor overestimates, and vice versa. In these cases, taking the average of the two would result in highly accurate predictions, which could be a fruitful route to explore. 

We will briefly explain a few important parameters that will be used in the following sections. Explanations of these parameters can also be found in the code comments.

    num_lookback_days - the number of prior days to take into account when trying to make a prediction about future days. When passed as a parameter to feature_engineering(), it will add data from prior days to every row for use with XGBRegressor. When passed as a parameter to run_lstm(), it will be passed as a parameter to the LSTM model, which will automatically look at the data in prior rows, thus eliminating the need for appending the data to every row. 
    
    num_days_to_predict - the number of days we are trying to predict
    
    num_days_difference - if 0, we will predict the actual target values; if 1, we will predict the difference of the target variable between one day and the next; can be set to numbers 2 and higher, but shouldn't be  
    
    size_test_set - number of days in the test set
    
    synthetic - only for use with the Arno River; number of sets of synthetic data to make
    
    batch_size - batch size for the initial training of the LSTM neural network
    
    num_epochs - number of epochs to run the initial training
    
    num_epochs_walk_forward - if walk_forward is True, the number of epochs to train on the most recently available data; this is a form of walk-forward validation 
    
    num_train_years_prior - if > 0, the number of this time period's prior years data to train on. For example, if the day is June 15, then we will train on data from June 8 - June 22 of the prior year, of two years prior, ... all the way up to num_train_years_prior. If 0, we will not train on data from prior years. This is a simplified attention mechanism, designed to capture and train on data that is similar from year to year. 
    
    num_epochs_train_prior_years - if num_train_years_prior > 0, the number of epochs we train on each set of prior years' data
    
    scale_y - if True, scale the target variable 
    
    walk_forward - if True, use walk-forward validation
    
    leak_data - if True, train on the most recently available data during walk-forward validation. This is useful for baseline benchmarking purposes, but should otherwise always be set to False because it leaks data
    
    rolling_sum - assuming we are predicting the difference between days, then if True, should calculate the actual predictions by taking the rolling sum - see invert_difference() for details 
    
    print_flags - if True, will print the stage and intermediate dataframes in feature_engineering(), making it clear exactly how the feature extraction is being executed



The predictions from both LSTM and XGBRegressor, along with the actual values, will be saved to files, and then loaded when making visualizations. This saves us from needing to re-train an entire model just to make a visualization. Please note that the interactive widget only works when running the notebook - when viewing a static version of the notebook, like on Kaggle, the widget does not appear, and the default prediction that is visualized is 7 days out from the current day. Due to this shortcoming, we also display a small dataframe with the MAE and RMSE for each of the `num_days_to_predict` days.

Note that when first executing a cell that plots results in a Kaggle environment, it is necessary to refresh the output of the working directory in order for the plot to appear.

In the last section, we demonstrate how to make real-world predictions with both LSTM and XGBRegressor. 

## Auser Aquifer

### Feature Importances

We begin by engineering our datasets as described in the "Feature Engineering" section. Since we have several `Temperature` variables available to us, we will choose one arbitrarily.

In [ ]:
auser_targets = ['Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_CoS', 'Depth_to_Groundwater_LT2']

name_of_temp_variable = 'Temperature_Orentano'
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Orentano'


name_of_target = 'Depth_to_Groundwater_SAL'
prepared_df_aquifer_Auser_SAL = df_aquifer_auser.copy()
prepared_df_aquifer_Auser_SAL, targets_Auser_SAL, original_target_values_Auser_SAL, \
        lookback_dict_Auser_SAL, target_dict_Auser_SAL = \
                                                feature_engineering(prepared_df_aquifer_Auser_SAL, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_CoS'
prepared_df_aquifer_Auser_CoS = df_aquifer_auser.copy()
prepared_df_aquifer_Auser_CoS, targets_Auser_CoS, original_target_values_Auser_CoS, \
        lookback_dict_Auser_CoS, target_dict_Auser_CoS = \
                                                feature_engineering(prepared_df_aquifer_Auser_CoS, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_LT2'
prepared_df_aquifer_Auser_LT2 = df_aquifer_auser.copy()
prepared_df_aquifer_Auser_LT2, targets_Auser_LT2, original_target_values_Auser_LT2, \
        lookback_Auser_dict_LT2, target_dict_Auser_LT2 = \
                                                feature_engineering(prepared_df_aquifer_Auser_LT2, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


Now that we have our feature-engineered datasets, let's run XGBRegressor to determine the most important features. We will choose the features that best predict the target variable 7 days in advance. This number is chosen because it seems like a reasonable prediction timeframe. However, any number less than `num_days_to_predict` can be used.

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Auser_SAL, features_Auser_SAL, y_test_Auser_SAL, y_pred_Auser_SAL = \
    run_xgb(prepared_df_aquifer_Auser_SAL, targets_Auser_SAL, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_SAL'
get_n_most_important_feature_indices(xgb_feature_importances_Auser_SAL, features_Auser_SAL, 
                                     name_of_target, num_most_important_features)

Unsurpisingly, the value of the variable `Depth_to_Groundwater_SAL` is the most important feature in predicting its own value in seven days. We see that the importances of other variables decreases drastically from there. Also unsurprisingly, we see that the `Hydrometry` variables are also very important. Interestingly, we also see that the `Depth_to_Groundwater_LT2` variable in March is important as well, perhaps for predicting values in that same month. Additionally, we also see several values of `Depth_to_Groundwater_SAL` up to 8 days prior to the date from which the prediction is being made (implying 15 days prior to the day that is being predicted) to be very important as well, featuring several times just in the 20 most important features. This shows the power of using feature extraction in the way that we did. We can see the non-linear effects that variables have on other variables. 

Scanning the remaining variables, it's clear that all of the different types of variables we extracted played a role in predicting the target variable. For this reason, we won't reduce the lookback period, nor eliminate any of the dummies and cross terms we made in `feature_engineering()`. 

Let's take a look at the results of running XGBRegressor on the other two targets. 

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Auser_CoS, features_Auser_CoS, y_test_Auser_CoS, y_pred_Auser_CoS = \
    run_xgb(prepared_df_aquifer_Auser_CoS, targets_Auser_CoS, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_CoS'
get_n_most_important_feature_indices(xgb_feature_importances_Auser_CoS, features_Auser_CoS, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Auser_LT2, features_Auser_LT2, y_test_Auser_LT2, y_pred_Auser_LT2 = \
    run_xgb(prepared_df_aquifer_Auser_LT2, targets_Auser_LT2, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_LT2'
get_n_most_important_feature_indices(xgb_feature_importances_Auser_LT2, features_Auser_LT2, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
# clear memory
del [prepared_df_aquifer_Auser_SAL, targets_Auser_SAL, original_target_values_Auser_SAL, \
        lookback_dict_Auser_SAL, target_dict_Auser_SAL,
    prepared_df_aquifer_Auser_CoS, targets_Auser_CoS, original_target_values_Auser_CoS, \
        lookback_dict_Auser_CoS, target_dict_Auser_CoS,
    prepared_df_aquifer_Auser_LT2, targets_Auser_LT2, original_target_values_Auser_LT2, \
        lookback_Auser_dict_LT2, target_dict_Auser_LT2]

gc.collect()

%memit

### Predictions

We will now use LSTM to make our predictions. For each day in the test set, we will make a prediction for the following thirty days. We will then plot the data with a widget that shows the differences between the real and predicted values for each of the 30 * 365 predictions in an easy-to-understand way. We will also print out the RMSE and MAE for the number of days in advance that we are trying to predict. These errors will be displayed in the bottom left corner of the plot. 

Please note that, currently, only the last widget plot whose cell was executed will have the slider to change the amount of days in advance the prediction is being made. The other plots will still display, but their sliders will not move. To change this, simply run the cell of the desired plot. Nothing is being recalculated, so the result should be displayed instantaneously. 

If this notebook is not currently running, and is being viewed in a static state, as it would be on Kaggle, the slider will not be available, and in its place, we have included a dataframe that contains all of the MAE and RMSE for all of the `num_days_to_predict` days.

Since the LSTM model automatically takes care of looking at past data (i.e. data in prior rows), we do not have to have all of the prior data appended to every row, so we set `num_lookback_days` to 0. Not including these lookback columns reduces the size of the dataframe by a factor of `num_lookback_days`. First, we will recreate the dataframes without the prior data appended.

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Orentano'

name_of_target = 'Depth_to_Groundwater_SAL'
prepared_df_aquifer_Auser_SAL = df_aquifer_auser.copy()
prepared_df_aquifer_Auser_SAL, targets_Auser_SAL, original_target_values_Auser_SAL, \
        lookback_dict_Auser_SAL, target_dict_Auser_SAL = \
                                                feature_engineering(prepared_df_aquifer_Auser_SAL, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_CoS'
prepared_df_aquifer_Auser_CoS = df_aquifer_auser.copy()
prepared_df_aquifer_Auser_CoS, targets_Auser_CoS, original_target_values_Auser_CoS, \
        lookback_dict_Auser_CoS, target_dict_Auser_CoS = \
                                                feature_engineering(prepared_df_aquifer_Auser_CoS, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_LT2'
prepared_df_aquifer_Auser_LT2 = df_aquifer_auser.copy()
prepared_df_aquifer_Auser_LT2, targets_Auser_LT2, original_target_values_Auser_LT2, \
        lookback_Auser_dict_LT2, target_dict_Auser_LT2 = \
                                                feature_engineering(prepared_df_aquifer_Auser_LT2, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


Now we can start training LSTM on our data, and make predictions.

In [ ]:
original_target = 'Depth_to_Groundwater_SAL'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 7
num_epochs_walk_forward = 2
num_train_years_prior = 8
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_Auser_SAL, y_test_actual_Auser_SAL, model_Auser_SAL = run_lstm(prepared_df_aquifer_Auser_SAL, 
            targets_Auser_SAL, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Auser_SAL, y_test_actual_Auser_SAL, 'Depth_to_Groundwater_SAL', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Auser_SAL, y_test_Auser_SAL, 'Depth_to_Groundwater_SAL', 'Meters', which_day_to_predict);

Notice that that we have `num_days_to_predict` more days in the XGBRegressor prediction - this is because of how we split our data into training and test sets.

In [ ]:
original_target = 'Depth_to_Groundwater_CoS'
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 7
num_epochs_walk_forward = 2
num_train_years_prior = 8
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False


df_pred_Auser_CoS, y_test_actual_Auser_CoS, model_Auser_CoS = run_lstm(prepared_df_aquifer_Auser_CoS, 
            targets_Auser_CoS, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Auser_CoS, y_test_actual_Auser_CoS, 'Depth_to_Groundwater_CoS', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Auser_CoS, y_test_Auser_CoS, 'Depth_to_Groundwater_Cos', 'Meters', which_day_to_predict);

In [ ]:
original_target = 'Depth_to_Groundwater_LT2'
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 7
num_epochs_walk_forward = 2
num_train_years_prior = 8
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_Auser_LT2, y_test_actual_Auser_LT2, model_Auser_LT2 = run_lstm(prepared_df_aquifer_Auser_LT2, 
            targets_Auser_LT2, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Auser_LT2, y_test_actual_Auser_LT2, 'Depth_to_Groundwater_LT2', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Auser_LT2, y_test_Auser_LT2, 'Depth_to_Groundwater_LT2', 'Meters', which_day_to_predict);

We can see that the prediction errors are relatively constant regardless of how many days in advance we are making a prediction, which is very good. 

In [ ]:
# clear memory
del [prepared_df_aquifer_Auser_SAL, targets_Auser_SAL, original_target_values_Auser_SAL, \
        lookback_dict_Auser_SAL, target_dict_Auser_SAL,
    prepared_df_aquifer_Auser_CoS, targets_Auser_CoS, original_target_values_Auser_CoS, \
        lookback_dict_Auser_CoS, target_dict_Auser_CoS,
    prepared_df_aquifer_Auser_LT2, targets_Auser_LT2, original_target_values_Auser_LT2, \
        lookback_Auser_dict_LT2, target_dict_Auser_LT2]

gc.collect()

%memit

## Doganella Aquifer

### Feature Importances

Recall that the data for Pozzo 3 starts becoming very inaccurate on 2019-09-19, so we truncate the data accordingly.

In [ ]:
doganella_targets = ['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2', 'Depth_to_Groundwater_Pozzo_3']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Monteporzio'


name_of_target = 'Depth_to_Groundwater_Pozzo_1'
prepared_df_aquifer_Doganella_Pozzo_1 = df_aquifer_doganella.copy()
prepared_df_aquifer_Doganella_Pozzo_1, targets_Doganella_Pozzo_1, original_target_values_Doganella_Pozzo_1, \
        lookback_dict_Doganella_Pozzo_1, target_dict_Doganella_Pozzo_1 = \
                                                feature_engineering(prepared_df_aquifer_Doganella_Pozzo_1, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_Pozzo_2'
prepared_df_aquifer_Doganella_Pozzo_2 = df_aquifer_doganella.copy()
prepared_df_aquifer_Doganella_Pozzo_2, targets_Doganella_Pozzo_2, original_target_values_Doganella_Pozzo_2, \
        lookback_dict_Doganella_Pozzo_2, target_dict_Doganella_Pozzo_2 = \
                                                feature_engineering(prepared_df_aquifer_Doganella_Pozzo_2, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_Pozzo_3'
prepared_df_aquifer_Doganella_Pozzo_3 = df_aquifer_doganella.copy().loc[
                                                df_aquifer_doganella['Date'] < '2019-09-19', :]
prepared_df_aquifer_Doganella_Pozzo_3, targets_Doganella_Pozzo_3, original_target_values_Doganella_Pozzo_3, \
        lookback_dict_Doganella_Pozzo_3, target_dict_Doganella_Pozzo_3 = \
                                                feature_engineering(prepared_df_aquifer_Doganella_Pozzo_3, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Doganella_Pozzo_1, features_Doganella_Pozzo_1, y_test_Doganella_Pozzo_1, y_pred_Doganella_Pozzo_1 = \
    run_xgb(prepared_df_aquifer_Doganella_Pozzo_1, targets_Doganella_Pozzo_1, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_Pozzo_1'
get_n_most_important_feature_indices(xgb_feature_importances_Doganella_Pozzo_1, features_Doganella_Pozzo_1, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Doganella_Pozzo_2, features_Doganella_Pozzo_2, y_test_Doganella_Pozzo_2, y_pred_Doganella_Pozzo_2 = \
    run_xgb(prepared_df_aquifer_Doganella_Pozzo_2, targets_Doganella_Pozzo_2, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_Pozzo_2'
get_n_most_important_feature_indices(xgb_feature_importances_Doganella_Pozzo_2, features_Doganella_Pozzo_2, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Doganella_Pozzo_3, features_Doganella_Pozzo_3, y_test_Doganella_Pozzo_3, y_pred_Doganella_Pozzo_3 = \
    run_xgb(prepared_df_aquifer_Doganella_Pozzo_3, targets_Doganella_Pozzo_3, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_Pozzo_3'
get_n_most_important_feature_indices(xgb_feature_importances_Doganella_Pozzo_3, features_Doganella_Pozzo_3, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_aquifer_Doganella_Pozzo_1, targets_Doganella_Pozzo_1, original_target_values_Doganella_Pozzo_1, \
        lookback_dict_Doganella_Pozzo_1, target_dict_Doganella_Pozzo_1,
    prepared_df_aquifer_Doganella_Pozzo_2, targets_Doganella_Pozzo_2, original_target_values_Doganella_Pozzo_2, \
        lookback_dict_Doganella_Pozzo_2, target_dict_Doganella_Pozzo_2,
    prepared_df_aquifer_Doganella_Pozzo_3, targets_Doganella_Pozzo_3, original_target_values_Doganella_Pozzo_3, \
        lookback_dict_Doganella_Pozzo_3, target_dict_Doganella_Pozzo_3]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Monteporzio'


name_of_target = 'Depth_to_Groundwater_Pozzo_1'
prepared_df_aquifer_Doganella_Pozzo_1 = df_aquifer_doganella.copy()
prepared_df_aquifer_Doganella_Pozzo_1, targets_Doganella_Pozzo_1, original_target_values_Doganella_Pozzo_1, \
        lookback_dict_Doganella_Pozzo_1, target_dict_Doganella_Pozzo_1 = \
                                                feature_engineering(prepared_df_aquifer_Doganella_Pozzo_1, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_Pozzo_2'
prepared_df_aquifer_Doganella_Pozzo_2 = df_aquifer_doganella.copy()
prepared_df_aquifer_Doganella_Pozzo_2, targets_Doganella_Pozzo_2, original_target_values_Doganella_Pozzo_2, \
        lookback_dict_Doganella_Pozzo_2, target_dict_Doganella_Pozzo_2 = \
                                                feature_engineering(prepared_df_aquifer_Doganella_Pozzo_2, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_Pozzo_3'
prepared_df_aquifer_Doganella_Pozzo_3 = df_aquifer_doganella.copy().loc[
                                                df_aquifer_doganella['Date'] < '2019-09-19', :]
prepared_df_aquifer_Doganella_Pozzo_3, targets_Doganella_Pozzo_3, original_target_values_Doganella_Pozzo_3, \
        lookback_dict_Doganella_Pozzo_3, target_dict_Doganella_Pozzo_3 = \
                                                feature_engineering(prepared_df_aquifer_Doganella_Pozzo_3, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
original_target = 'Depth_to_Groundwater_Pozzo_1'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 2
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_Doganella_Pozzo_1, y_test_actual_Doganella_Pozzo_1, model_Doganella_Pozzo_1 = \
            run_lstm(prepared_df_aquifer_Doganella_Pozzo_1, 
            targets_Doganella_Pozzo_1, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Doganella_Pozzo_1, y_test_actual_Doganella_Pozzo_1, 'Depth_to_Groundwater_Pozzo_1', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Doganella_Pozzo_1, y_test_Doganella_Pozzo_1, 'Depth_to_Groundwater_Pozzo_1', 'Meters', which_day_to_predict);

In [ ]:
original_target = 'Depth_to_Groundwater_Pozzo_2'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 2
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_Doganella_Pozzo_2, y_test_actual_Doganella_Pozzo_2, model_Doganella_Pozzo_2 = \
            run_lstm(prepared_df_aquifer_Doganella_Pozzo_2, 
            targets_Doganella_Pozzo_2, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Doganella_Pozzo_2, y_test_actual_Doganella_Pozzo_2, 'Depth_to_Groundwater_Pozzo_2', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Doganella_Pozzo_2, y_test_Doganella_Pozzo_2, 'Depth_to_Groundwater_Pozzo_2', 'Meters', which_day_to_predict);

In [ ]:
original_target = 'Depth_to_Groundwater_Pozzo_3'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 2
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_Doganella_Pozzo_3, y_test_actual_Doganella_Pozzo_3, model_Doganella_Pozzo_3 = \
            run_lstm(prepared_df_aquifer_Doganella_Pozzo_3, 
            targets_Doganella_Pozzo_3, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Doganella_Pozzo_3, y_test_actual_Doganella_Pozzo_3, 'Depth_to_Groundwater_Pozzo_3', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Doganella_Pozzo_3, y_test_Doganella_Pozzo_3, 'Depth_to_Groundwater_Pozzo_3', 'Meters', which_day_to_predict);

In [ ]:
del [prepared_df_aquifer_Doganella_Pozzo_1, targets_Doganella_Pozzo_1, original_target_values_Doganella_Pozzo_1, \
        lookback_dict_Doganella_Pozzo_1, target_dict_Doganella_Pozzo_1,
    prepared_df_aquifer_Doganella_Pozzo_2, targets_Doganella_Pozzo_2, original_target_values_Doganella_Pozzo_2, \
        lookback_dict_Doganella_Pozzo_2, target_dict_Doganella_Pozzo_2,
    prepared_df_aquifer_Doganella_Pozzo_3, targets_Doganella_Pozzo_3, original_target_values_Doganella_Pozzo_3, \
        lookback_dict_Doganella_Pozzo_3, target_dict_Doganella_Pozzo_3]

gc.collect()

%memit

## Luco Aquifer

### Feature Importances

In [ ]:
luco_targets = ['Depth_to_Groundwater_Podere_Casetta']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Monteroni_Arbia_Biena'


name_of_target = 'Depth_to_Groundwater_Podere_Casetta'
prepared_df_aquifer_Luco = df_aquifer_luco.copy()
prepared_df_aquifer_Luco, targets_Luco, original_target_values_Luco, \
        lookback_dict_Luco, target_dict_Luco = feature_engineering(prepared_df_aquifer_Luco, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_Luco, features_Luco, y_test_Luco, y_pred_Luco = \
    run_xgb(prepared_df_aquifer_Luco, targets_Luco, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_Podere_Casetta'
get_n_most_important_feature_indices(xgb_feature_importances_Luco, features_Luco, name_of_target, num_most_important_features)

Looking a bit further through the most important features, it seems like values from more than two weeks prior were more important than more recent values.

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_aquifer_Luco, targets_Luco, original_target_values_Luco, \
        lookback_dict_Luco, target_dict_Luco]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Monteroni_Arbia_Biena'


name_of_target = 'Depth_to_Groundwater_Podere_Casetta'
prepared_df_aquifer_Luco = df_aquifer_luco.copy()
prepared_df_aquifer_Luco, targets_Luco, original_target_values_Luco, \
        lookback_dict_Luco, target_dict_Luco = feature_engineering(prepared_df_aquifer_Luco, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


original_target = 'Depth_to_Groundwater_Podere_Casetta'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 3
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_Luco, y_test_actual_Luco, model_Luco = \
            run_lstm(prepared_df_aquifer_Luco, 
            targets_Luco, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_Luco, y_test_actual_Luco, 'Depth_to_Groundwater_Podere_Casetta', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_Luco, y_test_Luco, 'Depth_to_Groundwater_Podere_Casetta', 'Meters', which_day_to_predict);

In [ ]:
del [prepared_df_aquifer_Luco, targets_Luco, original_target_values_Luco, \
        lookback_dict_Luco, target_dict_Luco]

gc.collect()

%memit

## Petrignano Aquifer

### Feature Importances

In [ ]:
petrignano_targets = ['Depth_to_Groundwater_P24', 'Depth_to_Groundwater_P25']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Bastia_Umbra'


name_of_target = 'Depth_to_Groundwater_P24'
prepared_df_aquifer_petrignano_P24 = df_aquifer_petrignano.copy()
prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, original_target_values_petrignano_P24, \
        lookback_dict_petrignano_P24, target_dict_petrignano_P24 = \
                                                feature_engineering(prepared_df_aquifer_petrignano_P24, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_P25'
prepared_df_aquifer_petrignano_P25 = df_aquifer_petrignano.copy()
prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, original_target_values_petrignano_P25, \
        lookback_dict_petrignano_P25, target_dict_petrignano_P25 = \
                                                feature_engineering(prepared_df_aquifer_petrignano_P25, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_petrignano_P24, features_petrignano_P24, y_test_petrignano_P24, y_pred_petrignano_P24 = \
    run_xgb(prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_P24'
get_n_most_important_feature_indices(xgb_feature_importances_petrignano_P24, features_petrignano_P24, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_petrignano_P25, features_petrignano_P25, y_test_petrignano_P25, y_pred_petrignano_P25 = \
    run_xgb(prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_P25'
get_n_most_important_feature_indices(xgb_feature_importances_petrignano_P25, features_petrignano_P25, 
                                     name_of_target, num_most_important_features)

These results indicate that the two target variables are closely interlinked with one another. This is not surprising - if we look back at the values during the data cleaning stage, they look almost identical. This doesn't really tell us what we want to know, though. Let's rerun XGBRegressor, but when trying to predict one variable, we drop the other.

In [ ]:
petrignano_targets = ['Depth_to_Groundwater_P24', 'Depth_to_Groundwater_P25']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Bastia_Umbra'


name_of_target = 'Depth_to_Groundwater_P24'
prepared_df_aquifer_petrignano_P24 = df_aquifer_petrignano.copy().drop(columns=['Depth_to_Groundwater_P25'])
prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, original_target_values_petrignano_P24, \
        lookback_dict_petrignano_P24, target_dict_petrignano_P24 = \
                                                feature_engineering(prepared_df_aquifer_petrignano_P24, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_P25'
prepared_df_aquifer_petrignano_P25 = df_aquifer_petrignano.copy().drop(columns=['Depth_to_Groundwater_P24'])
prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, original_target_values_petrignano_P25, \
        lookback_dict_petrignano_P25, target_dict_petrignano_P25 = \
                                                feature_engineering(prepared_df_aquifer_petrignano_P25, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_petrignano_P24, features_petrignano_P24, y_test_petrignano_P24, y_pred_petrignano_P24 = \
    run_xgb(prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_P24'
get_n_most_important_feature_indices(xgb_feature_importances_petrignano_P24, features_petrignano_P24, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_petrignano_P25, features_petrignano_P25, y_test_petrignano_P25, y_pred_petrignano_P25 = \
    run_xgb(prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Depth_to_Groundwater_P25'
get_n_most_important_feature_indices(xgb_feature_importances_petrignano_P25, features_petrignano_P25, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, original_target_values_petrignano_P24, \
        lookback_dict_petrignano_P24, target_dict_petrignano_P24,
    prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, original_target_values_petrignano_P25, \
        lookback_dict_petrignano_P25, target_dict_petrignano_P25]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Bastia_Umbra'


name_of_target = 'Depth_to_Groundwater_P24'
prepared_df_aquifer_petrignano_P24 = df_aquifer_petrignano.copy()
prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, original_target_values_petrignano_P24, \
        lookback_dict_petrignano_P24, target_dict_petrignano_P24 = \
                                                feature_engineering(prepared_df_aquifer_petrignano_P24, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Depth_to_Groundwater_P25'
prepared_df_aquifer_petrignano_P25 = df_aquifer_petrignano.copy()
prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, original_target_values_petrignano_P25, \
        lookback_dict_petrignano_P25, target_dict_petrignano_P25 = \
                                                feature_engineering(prepared_df_aquifer_petrignano_P25, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
original_target = 'Depth_to_Groundwater_P24'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 8
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_petrignano_P24, y_test_actual_petrignano_P24, model_petrignano_P24 = \
            run_lstm(prepared_df_aquifer_petrignano_P24, 
            targets_petrignano_P24, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_petrignano_P24, y_test_actual_petrignano_P24, 'Depth_to_Groundwater_P24', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_petrignano_P24, y_test_petrignano_P24, 'Depth_to_Groundwater_P24', 'Meters', which_day_to_predict);

In [ ]:
original_target = 'Depth_to_Groundwater_P25'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 8
num_epochs_train_prior_years = 2
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_petrignano_P25, y_test_actual_petrignano_P25, model_petrignano_P25 = \
            run_lstm(prepared_df_aquifer_petrignano_P25, 
            targets_petrignano_P25, num_lookback_days, num_days_to_predict, size_test_set, num_days_difference, 
            original_target, scale_y, walk_forward, leak_data, synthetic, num_train_years_prior, batch_size, 
            num_epochs, num_epochs_walk_forward, num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_petrignano_P25, y_test_actual_petrignano_P25, 'Depth_to_Groundwater_P25', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_petrignano_P25, y_test_petrignano_P25, 'Depth_to_Groundwater_P25', 'Meters', which_day_to_predict);

In [ ]:
del [prepared_df_aquifer_petrignano_P24, targets_petrignano_P24, original_target_values_petrignano_P24, \
        lookback_dict_petrignano_P24, target_dict_petrignano_P24,
    prepared_df_aquifer_petrignano_P25, targets_petrignano_P25, original_target_values_petrignano_P25, \
        lookback_dict_petrignano_P25, target_dict_petrignano_P25]

gc.collect()

%memit

## Bilancino Lake

### Feature Importances

In [ ]:
lake_bilancino_targets = ['Lake_Level', 'Flow_Rate']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Le_Croci'


name_of_target = 'Lake_Level'
prepared_df_lake_bilancino_level = df_lake_bilancino.copy()
prepared_df_lake_bilancino_level, targets_lake_bilancino_level, original_target_values_lake_bilancino_level, \
        lookback_dict_lake_bilancino_level, target_dict_lake_bilancino_level = \
                                                feature_engineering(prepared_df_lake_bilancino_level, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate'
prepared_df_lake_bilancino_flow = df_lake_bilancino.copy()
prepared_df_lake_bilancino_flow, targets_lake_bilancino_flow, original_target_values_lake_bilancino_flow, \
        lookback_dict_lake_bilancino_flow, target_dict_lake_bilancino_flow = \
                                                feature_engineering(prepared_df_lake_bilancino_flow, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_lake_bilancino_level, features_lake_bilancino_level, y_test_lake_bilancino_level, \
    y_pred_lake_bilancino_level = run_xgb(prepared_df_lake_bilancino_level, targets_lake_bilancino_level, 
                                          test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Lake_Level'
get_n_most_important_feature_indices(xgb_feature_importances_lake_bilancino_level, features_lake_bilancino_level, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_lake_bilancino_flow, features_lake_bilancino_flow, y_test_lake_bilancino_flow, \
    y_pred_lake_bilancino_flow = run_xgb(prepared_df_lake_bilancino_flow, targets_lake_bilancino_flow, 
                                          test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate'
get_n_most_important_feature_indices(xgb_feature_importances_lake_bilancino_flow, features_lake_bilancino_flow, 

                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_lake_bilancino_level, targets_lake_bilancino_level, original_target_values_lake_bilancino_level, \
        lookback_dict_lake_bilancino_level, target_dict_lake_bilancino_level,
    prepared_df_lake_bilancino_flow, targets_lake_bilancino_flow, original_target_values_lake_bilancino_flow, \
        lookback_dict_lake_bilancino_flow, target_dict_lake_bilancino_flow]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Le_Croci'


name_of_target = 'Lake_Level'
prepared_df_lake_bilancino_level = df_lake_bilancino.copy()
prepared_df_lake_bilancino_level, targets_lake_bilancino_level, original_target_values_lake_bilancino_level, \
        lookback_dict_lake_bilancino_level, target_dict_lake_bilancino_level = \
                                                feature_engineering(prepared_df_lake_bilancino_level, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate'
prepared_df_lake_bilancino_flow = df_lake_bilancino.copy()
prepared_df_lake_bilancino_flow, targets_lake_bilancino_flow, original_target_values_lake_bilancino_flow, \
        lookback_dict_lake_bilancino_flow, target_dict_lake_bilancino_flow = \
                                                feature_engineering(prepared_df_lake_bilancino_flow, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
original_target = 'Lake_Level'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 30
num_epochs_walk_forward = 2
num_train_years_prior = 12
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_lake_bilancino_level, y_test_actual_lake_bilancino_level, model_lake_bilancino_level = \
            run_lstm(prepared_df_lake_bilancino_level, targets_lake_bilancino_level, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_lake_bilancino_level, y_test_actual_lake_bilancino_level, 'Lake_Level', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_lake_bilancino_level, y_test_lake_bilancino_level, 'Lake_Level', 'Meters', which_day_to_predict);

In [ ]:
original_target = 'Flow_Rate'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 12
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_lake_bilancino_flow, y_test_actual_lake_bilancino_flow, model_lake_bilancino_flow = \
            run_lstm(prepared_df_lake_bilancino_flow, targets_lake_bilancino_flow, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_lake_bilancino_flow, y_test_actual_lake_bilancino_flow, 'Flow_Rate', '$m^3 / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_lake_bilancino_flow, y_test_lake_bilancino_flow, 'Flow_Rate', '$m^3 / s$', which_day_to_predict);

In [ ]:
del [prepared_df_lake_bilancino_level, targets_lake_bilancino_level, original_target_values_lake_bilancino_level, \
        lookback_dict_lake_bilancino_level, target_dict_lake_bilancino_level,
    prepared_df_lake_bilancino_flow, targets_lake_bilancino_flow, original_target_values_lake_bilancino_flow, \
        lookback_dict_lake_bilancino_flow, target_dict_lake_bilancino_flow]

gc.collect()

%memit

## Amiata Water Spring

### Feature Importances

In [ ]:
water_spring_amiata_targets = ['Flow_Rate_Bugnano', 'Flow_Rate_Arbure', 'Flow_Rate_Ermicciolo', 'Flow_Rate_Galleria_Alta']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_S_Fiora'


name_of_target = 'Flow_Rate_Bugnano'
prepared_df_amiata_flow_rate_bugnano = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_bugnano, targets_amiata_flow_rate_bugnano, original_target_values_amiata_flow_rate_bugnano, \
        lookback_dict_amiata_flow_rate_bugnano, target_dict_amiata_flow_rate_bugnano = \
                                                feature_engineering(prepared_df_amiata_flow_rate_bugnano, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate_Arbure'
prepared_df_amiata_flow_rate_arbure = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_arbure, targets_amiata_flow_rate_arbure, original_target_values_amiata_flow_rate_arbure, \
        lookback_dict_amiata_flow_rate_arbure, target_dict_amiata_flow_rate_arbure = \
                                                feature_engineering(prepared_df_amiata_flow_rate_arbure, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate_Ermicciolo'
prepared_df_amiata_flow_rate_ermicciolo = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_ermicciolo, targets_amiata_flow_rate_ermicciolo, original_target_values_amiata_flow_rate_ermicciolo, \
        lookback_dict_amiata_flow_rate_ermicciolo, target_dict_amiata_flow_rate_ermicciolo = \
                                                feature_engineering(prepared_df_amiata_flow_rate_ermicciolo, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate_Galleria_Alta'
prepared_df_amiata_flow_rate_galleria_alta = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_galleria_alta, targets_amiata_flow_rate_galleria_alta, original_target_values_amiata_flow_rate_galleria_alta, \
        lookback_dict_amiata_flow_rate_galleria_alta, target_dict_amiata_flow_rate_galleria_alta = \
                                                feature_engineering(prepared_df_amiata_flow_rate_galleria_alta, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_amiata_flow_rate_bugnano, features_amiata_flow_rate_bugnano, y_test_amiata_flow_rate_bugnano, \
                            y_pred_amiata_flow_rate_bugnano = run_xgb(prepared_df_amiata_flow_rate_bugnano, \
                            targets_amiata_flow_rate_bugnano, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate_Bugnano'
get_n_most_important_feature_indices(xgb_feature_importances_amiata_flow_rate_bugnano, features_amiata_flow_rate_bugnano, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_amiata_flow_rate_arbure, features_amiata_flow_rate_arbure, y_test_amiata_flow_rate_arbure, \
                            y_pred_amiata_flow_rate_arbure = run_xgb(prepared_df_amiata_flow_rate_arbure, \
                            targets_amiata_flow_rate_arbure, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate_Arbure'
get_n_most_important_feature_indices(xgb_feature_importances_amiata_flow_rate_arbure, features_amiata_flow_rate_arbure, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_amiata_flow_rate_ermicciolo, features_amiata_flow_rate_ermicciolo, y_test_amiata_flow_rate_ermicciolo, \
                            y_pred_amiata_flow_rate_ermicciolo = run_xgb(prepared_df_amiata_flow_rate_ermicciolo, \
                            targets_amiata_flow_rate_ermicciolo, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate_Ermicciolo'
get_n_most_important_feature_indices(xgb_feature_importances_amiata_flow_rate_ermicciolo, features_amiata_flow_rate_ermicciolo, 
                                     name_of_target, num_most_important_features)

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_amiata_flow_rate_galleria_alta, features_amiata_flow_rate_galleria_alta, y_test_amiata_flow_rate_galleria_alta, \
                            y_pred_amiata_flow_rate_galleria_alta = run_xgb(prepared_df_amiata_flow_rate_galleria_alta, \
                            targets_amiata_flow_rate_galleria_alta, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate_Galleria_Alta'
get_n_most_important_feature_indices(xgb_feature_importances_amiata_flow_rate_galleria_alta, features_amiata_flow_rate_galleria_alta, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_amiata_flow_rate_bugnano, targets_amiata_flow_rate_bugnano, original_target_values_amiata_flow_rate_bugnano, \
        lookback_dict_amiata_flow_rate_bugnano, target_dict_amiata_flow_rate_bugnano,
    prepared_df_amiata_flow_rate_arbure, targets_amiata_flow_rate_arbure, original_target_values_amiata_flow_rate_arbure, \
        lookback_dict_amiata_flow_rate_arbure, target_dict_amiata_flow_rate_arbure,
    prepared_df_amiata_flow_rate_ermicciolo, targets_amiata_flow_rate_ermicciolo, original_target_values_amiata_flow_rate_ermicciolo, \
        lookback_dict_amiata_flow_rate_ermicciolo, target_dict_amiata_flow_rate_ermicciolo,
    prepared_df_amiata_flow_rate_galleria_alta, targets_amiata_flow_rate_galleria_alta, original_target_values_amiata_flow_rate_galleria_alta, \
        lookback_dict_amiata_flow_rate_galleria_alta, target_dict_amiata_flow_rate_galleria_alta]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_S_Fiora'


name_of_target = 'Flow_Rate_Bugnano'
prepared_df_amiata_flow_rate_bugnano = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_bugnano, targets_amiata_flow_rate_bugnano, original_target_values_amiata_flow_rate_bugnano, \
        lookback_dict_amiata_flow_rate_bugnano, target_dict_amiata_flow_rate_bugnano = \
                                                feature_engineering(prepared_df_amiata_flow_rate_bugnano, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate_Arbure'
prepared_df_amiata_flow_rate_arbure = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_arbure, targets_amiata_flow_rate_arbure, original_target_values_amiata_flow_rate_arbure, \
        lookback_dict_amiata_flow_rate_arbure, target_dict_amiata_flow_rate_arbure = \
                                                feature_engineering(prepared_df_amiata_flow_rate_arbure, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate_Ermicciolo'
prepared_df_amiata_flow_rate_ermicciolo = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_ermicciolo, targets_amiata_flow_rate_ermicciolo, original_target_values_amiata_flow_rate_ermicciolo, \
        lookback_dict_amiata_flow_rate_ermicciolo, target_dict_amiata_flow_rate_ermicciolo = \
                                                feature_engineering(prepared_df_amiata_flow_rate_ermicciolo, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

name_of_target = 'Flow_Rate_Galleria_Alta'
prepared_df_amiata_flow_rate_galleria_alta = df_water_spring_amiata.copy()
prepared_df_amiata_flow_rate_galleria_alta, targets_amiata_flow_rate_galleria_alta, original_target_values_amiata_flow_rate_galleria_alta, \
        lookback_dict_amiata_flow_rate_galleria_alta, target_dict_amiata_flow_rate_galleria_alta = \
                                                feature_engineering(prepared_df_amiata_flow_rate_galleria_alta, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
original_target = 'Flow_Rate_Bugnano'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 4
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_amiata_flow_rate_bugnano, y_test_actual_amiata_flow_rate_bugnano, model_amiata_flow_rate_bugnano = \
            run_lstm(prepared_df_amiata_flow_rate_bugnano, targets_amiata_flow_rate_bugnano, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_amiata_flow_rate_bugnano, y_test_actual_amiata_flow_rate_bugnano, 'Flow_Rate_Bugnano', '$Liters / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_amiata_flow_rate_bugnano, y_test_amiata_flow_rate_bugnano, 'Flow_Rate_Bugnano', '$Liters / s$', which_day_to_predict);

In [ ]:
original_target = 'Flow_Rate_Arbure'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 4
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_amiata_flow_rate_arbure, y_test_actual_amiata_flow_rate_arbure, model_amiata_flow_rate_arbure = \
            run_lstm(prepared_df_amiata_flow_rate_arbure, targets_amiata_flow_rate_arbure, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_amiata_flow_rate_arbure, y_test_actual_amiata_flow_rate_arbure, 'Flow_Rate_Arbure', '$Liters / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_amiata_flow_rate_arbure, y_test_amiata_flow_rate_arbure, 'Flow_Rate_Arbure', '$Liters / s$', which_day_to_predict);

In [ ]:
original_target = 'Flow_Rate_Ermicciolo'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 4
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_amiata_flow_rate_ermicciolo, y_test_actual_amiata_flow_rate_ermicciolo, model_amiata_flow_rate_ermicciolo = \
            run_lstm(prepared_df_amiata_flow_rate_ermicciolo, targets_amiata_flow_rate_ermicciolo, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_amiata_flow_rate_ermicciolo, y_test_actual_amiata_flow_rate_ermicciolo, 'Flow_Rate_Ermicciolo', '$Liters / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_amiata_flow_rate_ermicciolo, y_test_amiata_flow_rate_ermicciolo, 'Flow_Rate_Ermicciolo', '$Liters / s$', which_day_to_predict);

In [ ]:
original_target = 'Flow_Rate_Galleria_Alta'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 4
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_amiata_flow_rate_galleria_alta, y_test_actual_amiata_flow_rate_galleria_alta, model_amiata_flow_rate_galleria_alta = \
            run_lstm(prepared_df_amiata_flow_rate_galleria_alta, targets_amiata_flow_rate_galleria_alta, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_amiata_flow_rate_galleria_alta, y_test_actual_amiata_flow_rate_galleria_alta, 'Flow_Rate_Galleria_Alta', '$Liters / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_amiata_flow_rate_galleria_alta, y_test_amiata_flow_rate_galleria_alta, 'Flow_Rate_Galleria_Alta', '$Liters / s$', which_day_to_predict);

In [ ]:
del [prepared_df_amiata_flow_rate_bugnano, targets_amiata_flow_rate_bugnano, original_target_values_amiata_flow_rate_bugnano, \
        lookback_dict_amiata_flow_rate_bugnano, target_dict_amiata_flow_rate_bugnano,
    prepared_df_amiata_flow_rate_arbure, targets_amiata_flow_rate_arbure, original_target_values_amiata_flow_rate_arbure, \
        lookback_dict_amiata_flow_rate_arbure, target_dict_amiata_flow_rate_arbure,
    prepared_df_amiata_flow_rate_ermicciolo, targets_amiata_flow_rate_ermicciolo, original_target_values_amiata_flow_rate_ermicciolo, \
        lookback_dict_amiata_flow_rate_ermicciolo, target_dict_amiata_flow_rate_ermicciolo,
    prepared_df_amiata_flow_rate_galleria_alta, targets_amiata_flow_rate_galleria_alta, original_target_values_amiata_flow_rate_galleria_alta, \
        lookback_dict_amiata_flow_rate_galleria_alta, target_dict_amiata_flow_rate_galleria_alta]

gc.collect()

%memit

## Madonna di Canneto Water Spring

### Feature Importances

In [ ]:
water_spring_madonna_targets = ['Flow_Rate_Madonna_di_Canneto']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Settefrati'


name_of_target = 'Flow_Rate_Madonna_di_Canneto'
prepared_df_madonna_flow_rate = df_water_spring_madonna_di_canneto.copy()
prepared_df_madonna_flow_rate, targets_madonna_flow_rate, original_target_values_madonna_flow_rate, \
        lookback_dict_madonna_flow_rate, target_dict_madonna_flow_rate = \
                                                feature_engineering(prepared_df_madonna_flow_rate, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
# only six months of testing data, because of the anomalies
test_size = 180
which_day_to_predict = 7
xgb_feature_importances_madonna_flow_rate, features_madonna_flow_rate, y_test_madonna_flow_rate, \
                            y_pred_madonna_flow_rate = run_xgb(prepared_df_madonna_flow_rate, \
                            targets_madonna_flow_rate, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate_Madonna_di_Canneto'
get_n_most_important_feature_indices(xgb_feature_importances_madonna_flow_rate, features_madonna_flow_rate, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_madonna_flow_rate, targets_madonna_flow_rate, original_target_values_madonna_flow_rate, \
        lookback_dict_madonna_flow_rate, target_dict_madonna_flow_rate]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Settefrati'


name_of_target = 'Flow_Rate_Madonna_di_Canneto'
prepared_df_madonna_flow_rate = df_water_spring_madonna_di_canneto.copy()
prepared_df_madonna_flow_rate, targets_madonna_flow_rate, original_target_values_madonna_flow_rate, \
        lookback_dict_madonna_flow_rate, target_dict_madonna_flow_rate = \
                                                feature_engineering(prepared_df_madonna_flow_rate, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
original_target = 'Flow_Rate_Madonna_di_Canneto'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
# only six months of testing data, because of the anomalies
size_test_set = 180
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 2
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_flow_rate_madonna, y_test_actual_madonna, model_madonna = \
            run_lstm(prepared_df_madonna_flow_rate, targets_madonna_flow_rate, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_flow_rate_madonna, y_test_actual_madonna, 'Flow_Rate_Madonna_di_Canneto', '$Liters / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_madonna_flow_rate, y_test_madonna_flow_rate, 'Flow_Rate_Madonna_di_Canneto', '$Liters / s$', which_day_to_predict);

In [ ]:
del [prepared_df_madonna_flow_rate, targets_madonna_flow_rate, original_target_values_madonna_flow_rate, \
        lookback_dict_madonna_flow_rate, target_dict_madonna_flow_rate]

gc.collect()

%memit

## Lupa Water Spring

### Feature Importances

In [ ]:
water_spring_lupa_targets = ['Flow_Rate_Lupa']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = ''
no_temp = True

name_of_target = 'Flow_Rate_Lupa'
prepared_df_lupa_flow_rate = df_water_spring_lupa.copy()
prepared_df_lupa_flow_rate, targets_lupa_flow_rate, original_target_values_lupa_flow_rate, \
        lookback_dict_lupa_flow_rate, target_dict_lupa_flow_rate = \
                                                feature_engineering(prepared_df_lupa_flow_rate, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags, no_temp)


In [ ]:
# only 30 days of testing data, because of data structure (see "Data Cleaning" section on Lupa Water Spring)
test_size = 30
which_day_to_predict = 7
xgb_feature_importances_lupa_flow_rate, features_lupa_flow_rate, y_test_lupa_flow_rate, \
                            y_pred_lupa_flow_rate = run_xgb(prepared_df_lupa_flow_rate, \
                            targets_lupa_flow_rate, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Flow_Rate_Lupa'
get_n_most_important_feature_indices(xgb_feature_importances_lupa_flow_rate, features_lupa_flow_rate, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_lupa_flow_rate, targets_lupa_flow_rate, original_target_values_lupa_flow_rate, \
        lookback_dict_lupa_flow_rate, target_dict_lupa_flow_rate]

gc.collect()

%memit

### Predictions

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = ''
no_temp = True

name_of_target = 'Flow_Rate_Lupa'
prepared_df_lupa_flow_rate = df_water_spring_lupa.copy()
prepared_df_lupa_flow_rate, targets_lupa_flow_rate, original_target_values_lupa_flow_rate, \
        lookback_dict_lupa_flow_rate, target_dict_lupa_flow_rate = \
                                                feature_engineering(prepared_df_lupa_flow_rate, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags, no_temp)


In [ ]:
original_target = 'Flow_Rate_Lupa'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
# only 30 days of testing data, because of data structure (see "Data Cleaning" section on Lupa Water Spring)
size_test_set = 30
synthetic = 0
batch_size = 32
num_epochs = 2
num_epochs_walk_forward = 20
num_train_years_prior = 4
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_lupa_flow_rate, y_test_actual_lupa_flow_rate, model_lupa_flow_rate = \
            run_lstm(prepared_df_lupa_flow_rate, targets_lupa_flow_rate, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_lupa_flow_rate, y_test_actual_lupa_flow_rate, 'Flow_Rate_Lupa', '$Liters / s$', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_lupa_flow_rate, y_test_lupa_flow_rate, 'Flow_Rate_Lupa', '$Liters / s$', which_day_to_predict);

In [ ]:
del [prepared_df_lupa_flow_rate, targets_lupa_flow_rate, original_target_values_lupa_flow_rate, \
        lookback_dict_lupa_flow_rate, target_dict_lupa_flow_rate]

gc.collect()

%memit

## Arno River

### Feature Importances

In [ ]:
river_arno_targets = ['Hydrometry_Nave_di_Rosano']

num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Firenze'

name_of_target = 'Hydrometry_Nave_di_Rosano'
prepared_df_river_arno = df_river_arno_full_data.copy()
prepared_df_river_arno, targets_river_arno, original_target_values_river_arno, \
        lookback_dict_river_arno, target_dict_river_arno = \
                                                feature_engineering(prepared_df_river_arno, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)

# for demonstration with prediction
last_row_with_lookback_river_arno = prepared_df_river_arno.iloc[-1, :]
last_row_with_lookback_river_arno_features = last_row_with_lookback_river_arno[: -num_days_to_predict]
last_row_with_lookback_river_arno_targets = last_row_with_lookback_river_arno[ -num_days_to_predict:]

In [ ]:
test_size = 365
which_day_to_predict = 7
xgb_feature_importances_river_arno, features_river_arno, y_test_river_arno, \
                            y_pred_river_arno = run_xgb(prepared_df_river_arno, \
                            targets_river_arno, test_size, num_lookback_days, num_days_to_predict, which_day_to_predict)

In [ ]:
num_most_important_features = 20
name_of_target = 'Hydrometry_Nave_di_Rosano'
get_n_most_important_feature_indices(xgb_feature_importances_river_arno, features_river_arno, 
                                     name_of_target, num_most_important_features)

Since these dataframes consume a lot of memory, we will delete all of them. 

In [ ]:
del [prepared_df_river_arno, targets_river_arno, original_target_values_river_arno, \
        lookback_dict_river_arno, target_dict_river_arno]

gc.collect()

%memit

### Predictions

A lot of time was spent on this particular dataset. It should be noted that one can get very low error rates by running this model for a few epochs, without walk-forward validation, on very basic settings. However, this is only because the predictions stay relatively constant over time. Predictions with similar error rates, but that look much more similar to the actual values, can be achieved with more training. This demonstrates that MAE and RMSE do not capture the entire story that the data is telling. This is a particularly hard dataset to predict, but we have seen results more accurate than these, particularly when tuning the number of epochs run, as well as changing the LSTM model that is being used (multiple are given in the LSTM Code section). 

We will demonstrate two sets of predictions with LSTM: the first one will (probably) have lower MAE and RMSE, but will less accurately reflect the shape of the data; the second will (probably) have higher MAE and RMSE, but will more accurately reflect the shape of the data. Note the "probably" - neural networks have an element of randomness, and are not deterministic. 

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Firenze'

name_of_target = 'Hydrometry_Nave_di_Rosano'
prepared_df_river_arno = df_river_arno.copy()
prepared_df_river_arno, targets_river_arno, original_target_values_river_arno, \
        lookback_dict_river_arno, target_dict_river_arno = \
                                                feature_engineering(prepared_df_river_arno, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags, )


In [ ]:
original_target = 'Hydrometry_Nave_di_Rosano'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 1
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 5
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = False
leak_data = False
rolling_sum = False

df_pred_river_arno, y_test_actual_river_arno, model_river_arno = \
            run_lstm(prepared_df_river_arno, targets_river_arno, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_river_arno, y_test_actual_river_arno, 'Hydrometry_Nave_di_Rosano', 'Meters', num_days_in_advance=7)

Now let's try with synthetic data, walk-forward validation, and training on prior years.

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 5
print_flags = False
name_of_temp_variable = 'Temperature_Firenze_synthetic'

name_of_target = 'Hydrometry_Nave_di_Rosano_synthetic'
prepared_df_river_arno_synthetic = df_river_arno_full_data.copy()
prepared_df_river_arno_synthetic.columns += '_synthetic'
prepared_df_river_arno_synthetic.rename(columns={'Date_synthetic': 'Date'}, inplace=True)
prepared_df_river_arno_synthetic, targets_river_arno_synthetic, original_target_values_river_arno_synthetic, \
        lookback_dict_river_arno_synthetic, target_dict_river_arno_synthetic = \
                                                feature_engineering(prepared_df_river_arno_synthetic, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


Let's take a look at our dataset with synthetic data included.

In [ ]:
prepared_df_river_arno_synthetic

As we can see, we created an additional 10 years of data (synthetic = 5 -> 10 years of data, since we take the first two years of data and modify it 5 times).

In [ ]:
original_target = 'Hydrometry_Nave_di_Rosano_synthetic'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 5
batch_size = 1
num_epochs = 2
num_epochs_walk_forward = 2
num_train_years_prior = 5
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = True
leak_data = False
rolling_sum = False

df_pred_river_arno_synthetic, y_test_actual_river_arno_synthetic, model_river_arno_synthetic = \
            run_lstm(prepared_df_river_arno_synthetic, targets_river_arno_synthetic, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


In [ ]:
%matplotlib inline
plot_widget_results(df_pred_river_arno_synthetic, y_test_actual_river_arno_synthetic, 'Hydrometry_Nave_di_Rosano_synthetic', 'Meters', num_days_in_advance=7)

In [ ]:
%matplotlib inline
plot_xgb_results(y_pred_river_arno, y_test_river_arno, 'Hydrometry_Nave_di_Rosano', 'Meters', which_day_to_predict);

As noted in the introduction, forecasting large changes in the Arno River level is quite difficult. In one paper, the goal was simply to make accurate forecasts six hours in advance: https://www.researchgate.net/publication/248952964_Artificial_neural_network_approach_to_flood_forecasting_in_the_River_Arno. 

In [ ]:
del [prepared_df_river_arno, targets_river_arno, original_target_values_river_arno, \
        lookback_dict_river_arno, target_dict_river_arno]

del [prepared_df_river_arno_synthetic, targets_river_arno_synthetic, original_target_values_river_arno_synthetic, \
        lookback_dict_river_arno_synthetic, target_dict_river_arno_synthetic]

gc.collect()

%memit

# Future Predictions

Let's demonstrate how to predict the next 30 days from the last datapoint available for the Arno River. We will rerun the LSTM on the imputed dataset for the Arno River. 

We will train on all the data except the last `num_days_to_predict` days, so that we do not leak data into our model. 

Please note that this model is made to train as quickly as possible, and is not optimized for making the best predictions.

In [ ]:
# now num_lookback_days == 0 so that we don't create lookback variables in each row
num_lookback_days = 0
num_days_to_predict = 30
num_days_difference = 0
synthetic = 0
print_flags = False
name_of_temp_variable = 'Temperature_Firenze_example_prediction'

name_of_target = 'Hydrometry_Nave_di_Rosano_example_prediction'
prepared_df_river_arno_example_prediction = df_river_arno_imputed_data.copy()
prepared_df_river_arno_example_prediction.columns += '_example_prediction'
prepared_df_river_arno_example_prediction.rename(columns={'Date_example_prediction': 'Date'}, inplace=True)
prepared_df_river_arno_example_prediction, targets_river_arno_example_prediction, \
        original_target_values_river_arno_example_prediction, \
        lookback_dict_river_arno_example_prediction, target_dict_river_arno_example_prediction = \
                                                feature_engineering(prepared_df_river_arno_example_prediction, 
                                                name_of_target, name_of_temp_variable, num_lookback_days, 
                                                num_days_to_predict, num_days_difference, synthetic, print_flags)


In [ ]:
original_target = 'Hydrometry_Nave_di_Rosano_example_prediction'
# we have to reset the value of num_lookback_days here for the LSTM
num_lookback_days = 30
num_days_to_predict = 30
num_days_difference = 0
size_test_set = 365
synthetic = 0
batch_size = 32
num_epochs = 1
num_epochs_walk_forward = 2
num_train_years_prior = 5
num_epochs_train_prior_years = 1
scale_y = True
walk_forward = False
leak_data = False
rolling_sum = False

# only train up until the last num_days_to_predict days
df_pred_river_arno_example_prediction, y_test_actual_river_arno_example_prediction, model_river_arno_example_prediction = \
            run_lstm(prepared_df_river_arno_example_prediction.iloc[:-num_days_to_predict, :], targets_river_arno_example_prediction, num_lookback_days, 
            num_days_to_predict, size_test_set, num_days_difference, original_target, scale_y, walk_forward, 
            leak_data, synthetic, num_train_years_prior, batch_size, num_epochs, num_epochs_walk_forward, 
                     num_epochs_train_prior_years, rolling_sum)


Let's take a look at the predictions based on the imputed data.

In [ ]:
%matplotlib inline
plot_widget_results(df_pred_river_arno_example_prediction, y_test_actual_river_arno_example_prediction, 'Hydrometry_Nave_di_Rosano_example_prediction', 'Meters', num_days_in_advance=7)

Now, let's make predictions for the future.

In [ ]:
# load the model
saved_model = load_model('./LSTM_Models/LSTM_Hydrometry_Nave_di_Rosano_example_prediction.h5')
print(saved_model.summary())

In [ ]:
# Note: ideally, the scalers should have been returned by run_lstm()

X, y = prepared_df_river_arno_example_prediction.iloc[:, : -num_days_to_predict], \
       prepared_df_river_arno_example_prediction.iloc[:, -num_days_to_predict:]
X_scaled, X_scaler = scale_values(X, feature_range=(0, 1))
y_scaled, y_scaler = scale_values(y, feature_range=(0, 1))

num_lookback_days = 30
num_days_to_predict = 30

# only take last num_days_to_predict + 1 days
X_scaled_future = X_scaler.fit_transform(X[-(num_days_to_predict + 1):])

print('Shape of input to prediction: ', X_scaled_future.shape)

test_generator = TimeseriesGenerator(X_scaled_future, np.zeros(len(X_scaled_future)), length=num_lookback_days, batch_size=batch_size)

We see that the shape of the input to predict is (31, 1638) - this makes sense, since our LSTM model makes use of the last 30 days of data when making predictions. It is the last row of this input for which we are making a prediction.

In [ ]:
predicted_values_scaled = saved_model.predict(test_generator)
predicted_values = y_scaler.inverse_transform(predicted_values_scaled[:, :, 0])
river_arno_future_actual_values = df_river_arno.iloc[-num_days_to_predict:, -1].values
for i in range(len(predicted_values[0])):
    print('predicted hydrometry for {} days after {} is {}, actual value is {}'.format(
        i + 1, pd.to_datetime(df_river_arno_imputed_data['Date'].values[-1]), predicted_values[0][i], river_arno_future_actual_values[i]))

If we want make predictions using XGBRegressor, we can load the saved model, which was trained to predict the value 7 days from the date in the row being predicted. XGBRegressor can also be updated by calling `.fit()` and passing the parameter `xgb_model=` the currently loaded model. We will not re-train here on the imputed data, so we will only be predicting the value 7 days from the last available datapoint in `df_river_arno_full_data` after feature engineering.

In [ ]:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('./XGBregressor_Models/Hydrometry_Nave_di_Rosano_7_days_forward')  # load data
example_xgb_prediction_river_arno = bst.predict(xgb.DMatrix(last_row_with_lookback_river_arno_features.values.reshape(1, -1)))
print('predicted hydrometry for 7 days after {} is {}'.format(last_row_with_lookback_river_arno_features.name, example_xgb_prediction_river_arno[0]))


In [ ]:
del [prepared_df_river_arno_example_prediction, targets_river_arno_example_prediction, \
        original_target_values_river_arno_example_prediction, \
        lookback_dict_river_arno_example_prediction, target_dict_river_arno_example_prediction]

gc.collect()

In [ ]:
del bst
libc.malloc_trim(0)
gc.collect()

In [ ]:
%memit

# Future Work

Some areas for improvement:

    1) getting more data - e.g. weather (humidity, wind speed, air/water temperatures, etc.), energy consumption, proxies for water consumption
    2) implementing this: https://github.com/Arturus/kaggle-web-traffic/blob/master/how_it_works.md#working-with-long-timeseries in place of the current simplified attention mechanism
    3) implementing a more accurate synthetic data creation process
    4) abstracting some of the predictions into functions
    5) getting all the widgets to work at the same time, instead of only one at a time
    6) grouping LSTM functions into a class 
    7) making predictions 30 days in advance using XGBRegressor - we already have the code for this from the first attempt at solving this problem
    8) simplifying the output from getting the most important features using XGBRegressor